In [ ]:
pip install tiktoken
pip install bertopic
pip install polyglot
pip install PyICU
pip install pycld2
pip install fasttext
pip install langdetect
pip install tchan
pip install natasha
pip install pyLDAvis
pip install rutermextract
pip install rank_bm25

SyntaxError: invalid syntax (<ipython-input-3-a198bd754e2d>, line 2)

In [ ]:
import pandas as pd
import os
import random
import pickle
import pytz
from datetime import datetime
from collections import Counter
from tqdm import tqdm
import torch
import tiktoken
from bertopic.representation import OpenAI as bertopicOpenAI
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
#from lingua import Language, LanguageDetectorBuilder
from textblob import TextBlob
from polyglot.detect import Detector
import fasttext
import chardet
from langdetect import detect as langdetect_detect
import pandas as pd
import numpy as np
from langdetect import detect
from tchan import ChannelScraper
from transformers import pipeline, BertTokenizer, BertForSequenceClassification
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc,
)
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from rutermextract import TermExtractor

In [ ]:
import json

def json_read(file='wargonzo.json'):
# Potential solution: Use a try-except block to catch JSONDecodeError and identify problematic part
  try:
      # Opening JSON file
      with open(file, 'r') as f:
          tests = json.load(f)
  except json.JSONDecodeError as e:
      print(f"JSONDecodeError: {e}")
      # Additional suggestion: Check for common issues like file encoding or extra characters
      with open('wargonzo.json', 'r', encoding='utf-8') as f:
          content = f.read()
      print("Preview of file content around the error (if file size is manageable):")
      error_pos = e.pos if hasattr(e, 'pos') else 0
      start = max(error_pos - 50, 0)
      end = min(error_pos + 50, len(content))
      print(content[start:end])

  texts=[]
  for j in range(0, len(tests['messages'])):
      if isinstance(tests['messages'][j]['text'], str):
          for i in range(0, len(tests['messages'][j]['text'].split('\n'))):
              texts.append(tests['messages'][j]['text'].split('\n')[i].strip('\n').strip('⚡️').strip('🔥'))
      else:
          if isinstance(tests['messages'][j]['text'][0], dict):
              continue
          else:
              for i in range(0, len(tests['messages'][j]['text'][0].split('\n'))):
                  texts.append(tests['messages'][j]['text'][0].split('\n')[i].strip('\n').strip('⚡️').strip('🔥'))
  texts=list(filter(lambda x: x not in ['', ' '], texts))
  return texts

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Tue Mar 19 18:44:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
texts=[]
for t in ('wargonzo.json', 'opersvodki.json', 'rvvoenkorym.json', 'result.json'):
  text=json_read(t)
  texts=texts+text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
narratives=pd.read_excel('Projector2024_Narratives.xlsx', sheet_name='Narratives')
narratives.columns=(['id', 'source', 'link', 'sample', 'phrase', 'narrative'])
narratives.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'Index' object has no attribute '_format_flat'

   id source link sample                                             phrase  \
0   1   МОСТ    ?      ?  В России идёт частичная мобилизация, она затро...   
1   2   МОСТ    ?      ?         Россия воюет с НАТО, Америкой или Западом.   
2   3   МОСТ    ?      ?  НАТО, США или Запад угрожают России ядерным ор...   
3   4   МОСТ    ?      ?  Цель Запада — ослабить, разобщить и уничтожить...   
4   5   МОСТ    ?      ?   Россия не может сдаться, нужно воевать до конца.   

                                 narrative  
0  Дезинформация об успехах России в войне  
1                   Дискредитация "Запада"  
2                   Дискредитация "Запада"  
3                   Дискредитация "Запада"  
4        Оправдание вторжения РФ в Украину  

In [ ]:
theses=list(narratives.phrase.unique())
passages=texts[:]
passages

In [ ]:
len(passages)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


122627

https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Batches:   0%|          | 0/3833 [00:00<?, ?it/s]

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print("Top-5 lexical search (BM25) hits")
    for hit in bm25_hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-5 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-5 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))

    #return cross_inp

In [ ]:
search(query = "В России идёт частичная мобилизация, она затронет всего лишь 1% населения")

In [ ]:
search(query = "Россия воюет с НАТО, Америкой или Западом.")

In [ ]:
search(query = "Украинские пограничники не оказывают никакого сопротивления российским войскам")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Input question: Украинские пограничники не оказывают никакого сопротивления российским войскам
Top-5 lexical search (BM25) hits
	13.144	Срочно⚡️Аваков слил украинские гостайны российским пранкерам
	11.420	Ждут команды. Наши пограничники 
	11.336	В Бердычах и Победе идут ожесточённые встречные бои, ВСУ продолжают подтягивать туда резервы. Также российским войскам удалось продвинуться в Новомихайловке. 
	11.079	палестинского движения сопротивления:
	10.960	На Херсонско-Крымском направлении ВС РФ атаковали Крынки. Зачистить левобережье в истекшем году российским войскам не удалось. Но часть позиций отбили. Впрочем, ВСУ тоже не особо потеряли. Закрепились на островах. (Рис. 1)

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.858	На луганском направлении уже украинская армия пытается прорваться к трассе Р-66 в районе Краснопоповки и перерезать её участок к северу от Кременной. В свою очередь, российские войска атаковали в направлении Стельмаховки. (Рис. 5)
	0.855	На луганском 

In [ ]:
search(query = "Военная инфраструктура Украины выведена из строя")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Input question: Военная инфраструктура Украины выведена из строя
Top-5 lexical search (BM25) hits
	26.239	ФОТО⚡️Васильевская станция выведена из строя
	25.393	- Ладыжинская ТЭС выведена из строя (Винница) 
	18.794	Где-то на Авдеевском направлении бойцами из 1-го АК была выведена из строя, а после – уничтожена, очередная иностранная БМП. 
	16.025	В результате обстрела одна из ТЭС выведена из строя и прекратила производство электроэнергии. Кроме того было остановлено  водоснабжение по ряду адресов. Обошлось без жертв среди мирного населения.
	14.767	Россия полностью выведена из грузинской политической игры.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.850	Важно⚡️Ночные удары ВС РФ портят статистику ВСУ
	0.847	Россия нанесла ракетные удары по всей военной инфраструктуре Украины
	0.815	Катастрофа украинского самолета в ИРИ
	0.813	В Узбекистан не пускают граждан РФ из Афганистана
	0.813	– Организаторы авиакатастрофы всё сделали не профессионально.

------------------------

In [ ]:
search(query = "Успешное завершение «первого этапа спецоперации")

Input question: Успешное завершение «первого этапа спецоперации
Top-5 lexical search (BM25) hits
	10.918	Министр обороны Израиля назвал три этапа войны с ХАМАС
	10.363	В бою легче не будет, но полученные навыки обеспечат успешное выполнение задач.
	9.769	— Премьер Израиля Биньямин Нетаньяху объявил о старте третьего этапа войны с ХАМАС.
	9.275	На «Тавриде» овациями встретили героев спецоперации
	9.054	Ракетный налет США на Сирию происходил в 2 этапа. С интервалом в час. Вот надежное видео. 

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.771	Совсем скоро на нашем канале детальные подробности этой успешной операции.
	0.769	Лучший завтрак - это завтрак на свежезавоёванных украинских позициях. Даже прилёты 120-х мин не в состоянии повлиять на аппетит «спартанцев». Авдеевский котёл. Наши дни. 
	0.766	Завершим нашу серию диалогов спецкора проекта 
	0.755	Одна запрещенная соцсеть напомнила милый эпизод из прошлого.
	0.749	За прошедший месяц Рабочая группа внесла в Госдуму 

--

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
search(query = "Произошло «значительном снижении военного потенциала Украины»")

Input question: Произошло «значительном снижении военного потенциала Украины»


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Top-5 lexical search (BM25) hits
	13.033	«Кейс Зеленского. Новая история Украины».
	11.500	Премьера «Кейс Зеленского. Новая история Украины». 30 ноября 2023.
	9.927	Киев, разумеется, боится ватнического и откровенно пророссийский потенциала харьковчан, ведь первые очаги сопротивления майдановской власти разгорелись именно там. 
	9.505	- как всё произошло 
	9.505	Историческое событие произошло 

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.809	Залужный назначен послом в Великобритании
	0.802	Уволен за свою открытую политическую позицию, за поддержку нашей армии и спецоперации.
	0.780	Ранее министр обороны Швеции сделал заявление о том, что страна должна в кратчайшие сроки войти в состав НАТО для отражения «российской угрозы».
	0.776	Участники несанкционированной акции в Москве попытались затруднить работу вокзалов
	0.770	«Золотое кольцо» сжимается: положение украинских подразделений в городе Золотое становится критическим.

-------------------------

Top-5 Cross-Encoder

In [ ]:
dat={}
for i in theses:
  hits=search(query = i)
  dat[i]=hits

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Input question: В России идёт частичная мобилизация, она затронет всего лишь 1% населения.
Top-5 lexical search (BM25) hits
	26.034	‼️⚡️В России объявлена частичная мобилизация, указ подписан — Путин
	25.253	• Частичная мобилизация почти заканчивается, примерно за две недели она будет завершена;
	24.888	Владимир Путин: Частичная мобилизация завершена
	22.930	Частичная мобилизация в Подмосковье завершена, сообщил губернатор области.
	19.689	Вполне ожидаемые заголовки западных СМИ. У нас частичная мобилизация, у них всеобщая паника😁

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.780	Телеканалы RTL провели опрос: «Стоит ли Украине отдать часть своей территории России для достижения мира?». И большинство ответили утвердительно (47%), ещё 41% посчитали, что это не решение проблемы, и 12% отказались отвечать.
	0.779	• мероприятия по частичной мобилизации в России завершены;
	0.775	• США начали выдавать визы российской делегации во главе с Лавровым для участия в заседаниях Сов

In [ ]:
Input question: В России идёт частичная мобилизация, она затронет всего лишь 1% населения.
Top-5 lexical search (BM25) hits
	26.034	‼️⚡️В России объявлена частичная мобилизация, указ подписан — Путин
	25.253	• Частичная мобилизация почти заканчивается, примерно за две недели она будет завершена;
	24.888	Владимир Путин: Частичная мобилизация завершена
	22.930	Частичная мобилизация в Подмосковье завершена, сообщил губернатор области.
	19.689	Вполне ожидаемые заголовки западных СМИ. У нас частичная мобилизация, у них всеобщая паника😁

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.780	Телеканалы RTL провели опрос: «Стоит ли Украине отдать часть своей территории России для достижения мира?». И большинство ответили утвердительно (47%), ещё 41% посчитали, что это не решение проблемы, и 12% отказались отвечать.
	0.779	• мероприятия по частичной мобилизации в России завершены;
	0.775	• США начали выдавать визы российской делегации во главе с Лавровым для участия в заседаниях Совбеза ООН.
	0.774	Россия чётко обозначает, что текущие границы военной операции могут измениться только в сторону расширения.
	0.773	Бои за Лисичанск: подразделения Народной милиции ЛНР атакуют с восточной стороны.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.518	Россия чётко обозначает, что текущие границы военной операции могут измениться только в сторону расширения.
	8.468	"Легко сказать, что опасения не обоснованы, но лица, принимающие решения, должны быть готовы к риску эскалации", - заявил Ниинистё в четверг на встрече с членами Ассоциации политических журналистов.
	8.433	Во время мигрантского кризиса 2015 года Швеция приняла наибольшее число беженцев в перерасчёте на душу населения. В шведском докладе сказано, что нетто-затраты на мигрантов и их родственников составляют $10 миллиардов. Средние нетто-затраты составляют 2.38% ВВП.
	8.355	Частичная мобилизация в Подмосковье завершена, сообщил губернатор области.
	8.338	Россия – одна из немногих стран, отправившая людей в Сирию для помощи после землетрясения.

Input question: Россия воюет с НАТО, Америкой или Западом.
Top-5 lexical search (BM25) hits
	24.565	• Россия воюет не столько с украинской армией, сколько с коллективным Западом;
	15.953	Россия не воюет против талибов, она воюет за Рахмона, и за всю власть.
	15.382	При том, что Россия воюет с сатанистами, которых сделали из русских людей.
	14.679	Вместе с Украиной Россия демилитаризует и НАТО
	14.676	• НАТО останется региональным альянсом, мы не собираемся расширятся на Азию, это Альянс между Америкой и Европой.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.876	• Россия воюет не столько с украинской армией, сколько с коллективным Западом;
	0.836	Наступление на Славянск и Краматорск: подразделения Народной милиции ЛНР закрепились в Заречном и штурмуют Ямполь.
	0.828	🤦Украинские ведьмы записали видео с проклятиями в адрес России и русских
	0.824	Им не нужна Россия. Она нужна нам - Путин о Западе
	0.824	Ну так то Англия. Там оно все ради свободы и счастья всех людей на земле. А не упаси Бог за геополитику, не за нефть или финансовые интересы какие.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.416	Напомню, что в начале СВО западные СМИ писали, что русская армия возьмёт Киев и решит все задачи буквально за две недели. Это было приятно слышать многим.
	9.327	Во время землетрясения на Гаити я сутки работал с американскими спасателями. Я был поражен тем, что американцы не лезут внутрь завалов. За них это делают волонтеры, которым платят соответствующую зарплату.
	9.240	"Западные народы имеют совершенно неверную картину ситуации. Россия могла бы уничтожить Украину за день без использования ядерного оружия", — написал Робертс на своем сайте.
	9.220	Ну так то Англия. Там оно все ради свободы и счастья всех людей на земле. А не упаси Бог за геополитику, не за нефть или финансовые интересы какие.
	9.157	Что перспектива горизонтальной, на другие страны региона, эскалации выглядит самой реальной за десятилетия.
Input question: НАТО, США или Запад угрожают России ядерным оружием.
Top-5 lexical search (BM25) hits
	24.604	США заподозрили Россию в экспериментах с ядерным оружием.
	19.167	КНДР спустила на воду подлодку с тактическим ядерным оружием
	18.624	3. нанесение удара по критической инфраструктуре, которая управляет ядерным оружием;
	16.884	В них, по данным источников, примут участие части, оснащенные новым ядерным оружием, а также традиционно будет задействован терминал "Чегет" системы "Казбек" - называемый "ядерным чемоданчиком".
	16.099	Так что утверждение США о том, что им угрожают «ядерный потенциал» России и Китая не соответствует действительности.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.846	🤡Зеленский предложил Западу нанести ядерный удар по России:
	0.837	 А если старейшины допустят джихад, мы скоро завоюем Иран".
	0.835	Нападавший был задержан.
	0.835	Уже завтра первая серия – об артиллерии. Нашей, не той под которую попала сегодня наша команда с
	0.835	США заподозрили Россию в экспериментах с ядерным оружием.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.241	Иллюзий не нужно. США опасный противник. Но не тот, что раньше. Майк Тайсон очень опасный боксёр до сих пор. Втащит раз, мало не покажется. Но он не может драться 12 раундов. Не выстроит. Против молодого и злого.
	9.171	Начитавшись в ленте свежих  репортажей про ужас и моральный террор в МГД, напоминаю на всякий случай, что такое хорошо и как на самом деле надо.
	9.074	«Наши программисты лучшие в мире и все это знают. К сожалению, они сейчас уезжают за рубеж, думая, что могут попасть в армию и пойти служить с оружием в руках. Но их оружие – компьютер», - заявил он.
	9.061	Сержант Воробьев не растерялся, прыгнул за руль машины, отогнал ее за расположение и умудрился потушить возгорание. Обыкновенный подвиг. Без сомнения заслуживающий боевой награды.
	9.057	США ожидают, что Киев в ближайшие недели проведёт контрнаступление, заявила американский постпред при НАТО Джулиан Смит.
Input question: Цель Запада — ослабить, разобщить и уничтожить Россию.
Top-5 lexical search (BM25) hits
	60.373	• Цель Запада — ослабить, разобщить и уничтожить Россию.
	41.954	• Цель США — ослабить, разобщить и уничтожить в итоге Россию, используя марионеточный режим в Киеве как таран;
	27.229	• США хотят ослабить Россию, но их конечная цель — Китай;
	24.526	• Попытки ряда государств переписать мировую историю становятся все агрессивнее, их цель — ослабить Россию;
	18.254	Всякое желание уничтожить Россию будет означать конец мира — Патриарх Кирилл

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.992	• Цель Запада — ослабить, разобщить и уничтожить Россию.
	0.814	"Запад ежедневно, с пеной у рта повторяет заклинания: "Даем Украине еще оружия"; "Даем дальнобойные ракеты, крылатые, бескрылые, танки"; "Скоро будут самолеты, но не сейчас, чуть позже"; "Надо добить Россию", - заявил Лавров в интервью телеканалу "Царьград".
	0.807	ЗЫ: вообще-то, учетная ставка в 20% считается в нынешних условиях "заградительной", этого даже ЦБ не отрицает.
	0.797	• Цель США — ослабить, разобщить и уничтожить в итоге Россию, используя марионеточный режим в Киеве как таран;
	0.786	Центральноазиатским политикам стоит очень серьёзно задуматься и напрячься.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.773	• Цель США — ослабить, разобщить и уничтожить в итоге Россию, используя марионеточный режим в Киеве как таран;
	8.669	Запад продолжает нагнетать ситуацию на Украине, ставя целью добить Россию, заявил глава МИД РФ Сергей Лавров.
	8.593	Заявить, что расследование будет тщательным, пообещать, что будут учтены все детали, что ни национальные, ни местные, ни корпоративные, ни клановые интересы не повлияют на результат. Раз дело вызывает такой резонанс, то надо ответить на вопросы общества.
	8.581	Я вообще-то ждал, что она за цитирование Ивана Ильина запишет и президента нашего в фашисты, но нет. Не рискнула. Осторожность, видать, не позволила.
	8.554	• Цель Запада — ослабить, разобщить и уничтожить Россию.
Input question: Россия не может сдаться, нужно воевать до конца.
Top-5 lexical search (BM25) hits
	22.304	Воевать против них до конца -
	17.945	Может просто сразу сдаться? Ну просто сдаться и всё.
	17.007	Многое ему удалось сделать, многое доводить до конца нужно нам с Вами..
	16.283	Кто не жил в США, не может до конца понять всю чудовищность бездны американского среднего образования.
	16.281	Не до конца завьюженный

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.847	Подтверждается ситуация с таможней
	0.845	Русские сражаются до конца.
	0.838	• Россия сожалеет, что ООН не осудила Киев за удары по Донецку;
	0.834	С днем рождения, Донецк!
	0.818	Мне кажется, что это должно понравиться борцам с йогой.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.133	Полиция отказывается возбуждать дело, несмотря на собранные доказательства, и на то, что этому скандалу посвятил небольшое расследование журнал Forbes. Ужасы режима как есть.
	8.864	Ситуация с каналом "Дождь", который получил лицензию на европейское вещание, означает, что Запад взял однозначный курс на усиление информационной войны против России. Точнее не так, не совсем против России.
	8.792	И разумеется, на сайте каждого ведомства должна быть поминальная страница, посвященная сотрудникам.
	8.746	Ситуация в Авдеевке и Марьинке контролируемая, опасаться серьезного продвижения ВСУ не стоит.
	8.594	Сражения под Донецком: подразделения Народной милиции ДНР наступают в районе Невельского.
Input question: Те, кто критикует российские войска или государство - предатели.
Top-5 lexical search (BM25) hits
	17.520	Спасибо тем, кто нас читает, критикует, одобряет и упоминает.
	15.832	Да, есть те, кто читает российский сегмент Телеграма или российские новостные сайты. Но их мало, потому что люди просто боятся.
	14.520	Оказывается, те, кто говорят, что они не российские, а русские политики, как Петр Толстой
	14.514	И еще раз... Враги не те, кто почему-то думает иначе, чем вы. А те, кто подбрасывает наркотики. И наказаны должны быть не те, кто работает в неправильных изданиях или что-то не так говорит, а те, кто фабрикует уголовные дела. Запомните это, пожалуйста.
	14.448	Не идиоты. Предатели. Предатели жизненных интересов армянского народа.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.859	Смотрите эксклюзивные кадры высадки в Гостомеле, снятые российскими десантниками.
	0.856	Контратака на Харьковском направлении: российские войска отодвинули ВСУ от границы.
	0.852	Кадры последствий ракетного удара ВСУ по Арабатской стрелке в Херсонской области.
	0.851	И наконец, Киев – просто русский город, в котором всегда думали и говорили по-русски.
	0.850	После выхода на административные границы ЛНР союзные войска приступили к освобождению территорий Донецкой Республики.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.667	Оказывается, те, кто говорят, что они не российские, а русские политики, как Петр Толстой
	8.579	Так вот после Кипра два члена НАТО в состоянии тихой войны.  То вертолёт собьют, то самолёт, в год десятки провокаций.
	8.507	Криворожское направление: российские войска атакуют в районе Высокополья и Потёмкина.
	8.506	Так вот, судя по отсутствии какой-либо реакции, органы госбезопасности в стране отсутствуют.
	8.501	Российские войска развивают наступление в Марьинке. Русская пехота ведёт бои на центральных улицах города.
Input question: Российские войска используют высокоточное оружие для ударов только по военным объектам.
Top-5 lexical search (BM25) hits
	23.981	Видео⚡️ВСУ используют беспилотники для повторных ударов по спасателям
	23.016	Действия миномётного расчета, выполняющего задачи по нанесению ударов по военным объектам, технике и живой силе противника.
	22.927	Украина использует только оружие собственного производства для ударов по объектам на территории России, заявил в четверг секретарь украинского Совета нацбезопасности и обороны (СНБО) Алексей Данилов.
	22.786	• США поддерживают нанесение Украиной ударов по военным объектам в Крыму, считают их "законными целями";
	21.974	Как ВСУ вхолостую применяют высокоточное оружие НАТО

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.914	 Ранним утром пресечена попытка киевского режима атаковать беспилотниками объекты на территории Тульской области.
	0.904	• ВКС России поразили высокоточными ракетами семь военных объектов Украины, в том числе три склада ракетного вооружения и боеприпасов;
	0.901	Ночью российской армией были нанесены удары по украинским военным объектам в Запорожье, г.Чугуев Харьковской области и трех районах Днепропетровской области.
	0.900	Утренние удары ВС России по объектам ВСУ в Константиновке Донецкой Народной Республики.
	0.900	Вооруженные Силы России нанесли ракетный удар высокоточным оружием по объектам военной инфраструктуры Украины.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	7.999	 Ранним утром пресечена попытка киевского режима атаковать беспилотниками объекты на территории Тульской области.
	7.893	• Ракетные войска поразили украинский объединенный штаб обороны Артемовска, а также подразделения иностранного легиона в Константиновке
	7.851	В Киеве после ударов по российским аэродромам под Саратовом, Рязанью и Курском угрожают атаками на объекты, в том числе, в Сибири — FT со ссылкой на "неназванного советника украинских властей по вопросам обороны"
	7.729	 Кадры попадания российской ракеты по военному объекту и последующего взрыва в Киеве.
	7.696	Результат уничтожения укрепрайона ВСУ фиксировался средствами объективного контроля.
Input question: Российские войска не атакуют мирное население.
Top-5 lexical search (BM25) hits
	20.839	Наступление на Запорожье: российские войска атакуют Гуляйполе.
	20.839	Запорожский фронт: российские войска атакуют под Угледаром.
	20.213	Харьковский фронт: российские войска атакуют на двух направлениях.
	19.624	Харьковский фронт: российские войска атакуют в районе Старого Салтова.
	19.068	Криворожское направление: российские войска атакуют в районе Высокополья и Потёмкина.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.884	Русские нарубились с Русскими за многие века нашей великой Истории.
	0.857	Эксклюзив⚡️Бородай расскал о наступлении ВСУ на Торское
	0.854	На луганском направлении российские войска атаковали в сторону Макеевки и в Новосёловском. (Рис. 4)
	0.849	Криворожское направление: российские войска атакуют в районе Высокополья и Потёмкина.
	0.844	Украинские войска перешли в наступление в направлении Новая Каменка — Берислав в Херсонской области — власти региона

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.068	Российские войска активно наступают на купянском направлении и продолжают наступление на авдеевском и марьинском, идут ожесточенные бои — замминистра обороны Украины Анна Маляр
	9.040	На донецком фронте российские войска атакуют со стороны Новомихайловки. Цель – выйти к Константиновке и таким образом отрезать участок рокады Угледар-Марьинка. По укрепрайону ВСУ в последнем из перечисленных населённых пунктов работала артиллерия. Такая же картина и в Красногоровке. Северо-восточнее российские войска продвинулись у Невельского. (Рис.3)
	9.024	На луганском направлении очередная атака российских войск на местную Белогоровку. Севернее – попытка движения на Терны и Макеевку. (Рис. 4)
	8.995	Борьба тактик: российские войска и Народная милиция ДНР и ЛНР возобновили наступление на Славянск, украинская армия пытается выйти в тыл Изюмской группировке с севера.
	8.981	На соледарском участке российские войска атакуют с северо-востока. Штурмовые действия они ведут в районе Весёлого и Белогоровки. Внимание, это не Белогоровка на территории Луганской области. Цель атаки – перерезать рокаду на этом участке фронта. (Рис. 2)
Input question: Это не война, а спецоперация или специальная военная операция (СВО).
Top-5 lexical search (BM25) hits
	30.475	Вот почему началась Специальная военная операция.
	28.644	Главный акцент сейчас - специальная военная операция, отметил он.
	25.572	Специальная военная операция продолжится до достижения поставленных целей — министр обороны РФ
	24.905	• Специальная военная операция - вынужденная и необходимая мера, надо быть благодарными российским воинам;
	23.656	И да у нас специальная операция. Но - это ключевое слово - военная. Так что удары по мостам, нефтебазам, железнодорожным узлам, это залог успеха СВО. Равно как отсутствие ударов залог другого исхода.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.828	Ситуация в Авдеевке и Марьинке контролируемая, опасаться серьезного продвижения ВСУ не стоит.
	0.820	Опасаясь эскалации со стороны Ливана, армия Израиля продолжает стягивать войскам к северной границе.
	0.816	Главный акцент сейчас - специальная военная операция, отметил он.
	0.812	Эвакуация по-европейски: спецоперация ОБСЕ в Донецке
	0.810	Вот почему началась Специальная военная операция.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.756	Умеренная  оппозиция, читай исламские боевики, воевали против Асада. А это по умолчанию индульгенция за все преступления.
	8.543	Главное сейчас, это стабильная внутрення ситуация, а армия свою работу сделает.
	8.543	Главное сейчас, это стабильная внутрення ситуация, а армия свою работу сделает.
	8.486	Опасаясь эскалации со стороны Ливана, армия Израиля продолжает стягивать войскам к северной границе.
	8.405	Вопрос ведь, собственно, не в военкорах и военблогерах, как явлении, или как элементе влияния на процессы (а они да, элемент влияния, даже если кому-то это не нравится). Это все вторично.
Input question: США, Запад и НАТО хотят развалить Россию.
Top-5 lexical search (BM25) hits
	18.023	"Очевидно, что США хотят победить Россию … за счет ЕС
	17.527	• США хотят ослабить Россию, но их конечная цель — Китай;
	15.906	• США и НАТО хотят победы над Россией "на поле боя" для уничтожения нашей страны.
	15.078	• США готовы "встать рядом" с союзниками по НАТО, но пока не хотят обсуждать гипотетические сценарии;
	13.181	• Запад пытается уничтожить и расчленить Россию, лишить самостоятельности республики бывшего СССР.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.841	США считают запрет на ввоз зерна из Украины суверенным решением стран Европы, сообщает  представиль Белого  дома.
	0.826	На Западе в политике работает бизнес-мышление. Не работает этот проект, надо его исправить, не может это государство решить задачи, значит нужно слияние-поглощение.
	0.820	Народ сам всё сказал, сам решил. У людей простой подход и честный: наплевать и забыть.
	0.809	США возлагают на Россию ответственность за ракетный инцидент в Польше вне зависимости от того, какими будут результаты расследования — Белый дом.
	0.807	США считают запрет на ввоз зерна из Украины суверенным решением стран Европы – Белый дом

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.073	США считают запрет на ввоз зерна из Украины суверенным решением стран Европы, сообщает  представиль Белого  дома.
	9.061	На Западе в политике работает бизнес-мышление. Не работает этот проект, надо его исправить, не может это государство решить задачи, значит нужно слияние-поглощение.
	9.045	И конечно, не могу не поразиться, сколь умелые ребята на Западе. Дело даже не в том, что возникает странная схема. Польша работает против России, а Сорос против Польши (это к вопросу о том, существуют ли у кого-либо постоянные союзники).
	9.043	США возлагают на Россию ответственность за ракетный инцидент в Польше вне зависимости от того, какими будут результаты расследования — Белый дом.
	8.902	Турки, Израиль, Азербайджан, а за спиной у них США. Ну то есть как "за спиной". Воевать каждый будет за свой интерес. Точнее будет так думать, а, в конечном счёте, за интересы США.
Input question: Россия освобождает Украину от нацистов.
Top-5 lexical search (BM25) hits
	15.410	• Норвегия пригласит Украину вместо России на 80-летие освобождения своей провинции от нацистов — издание Verdens gang
	15.192	❗️С каждым днём Российская армия освобождает населенные пункты Украины от подразделений укронацистов.
	14.795	"Мы не бунтари, не партизаны, мы просто освобождает страну от оккупантов и коллаборционистов".
	14.258	Израиль освобождает палестинских женщин и подростков
	13.396	• Госдума приняла закон, который освобождает туроператоров от налога на добавленную стоимость при продаже путевок по России

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.866	Последствия обстрелов украинскими нацистами Горловки.
	0.858	В Нижегородской области очередная интересная новация -
	0.858	Очередная быстрая реакция нижегородских силовиков на противоправные действия иностранца.
	0.857	Единственное общее у всех режимов на Украине - насильственная украинизация населения.
	0.848	Последствия обстрела со стороны ВСУ Петровского района г Донецк.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.321	Россия не только нарастила производство своего вооружения, но и неплохо изучила вражеское оружие, узнав для себя много полезного, рассказал замглавы Совета безопасности РФ Дмитрий Медведев в статье для журнала "Национальная оборона".
	9.229	Россия ограничивает использование своего неба для британских самолетов, включая транзит — Росавиация.
	9.112	Ситуация на линии соприкосновения обостряется. Есть мнение, что нацбаты начали обстрелы с раннего утра. НМ ДНР открыла ответный огонь на подавление.
	9.094	• Россия ожидает от международных организаций принципиальных оценок о несоблюдении Украиной гуманитарного права.
	9.040	Россия, наконец-то, в этом треше не участвовала, но просьбу украинской группы удовлетворила как могла.
Input question: Украиной правят нацисты, фашисты или бандеровцы.
Top-5 lexical search (BM25) hits
	12.989	фашисты утюжили землю
	12.372	Или как рассказ, про то, что Борман жил в Аргентине под чужим именем. Ну или про то, что на самом деле миром правят банкиры, вступившие в  сговор с инопланетянами.
	11.780	Бандеровцы в российской власти и госизмена
	11.744	Эксклюзив⚡️«Братья Путина»: Фашисты гибнут, люди живут
	11.631	При отступлении из Волновахи, бандеровцы добивали своих раненых сослуживцев в больнице. У всех раненых - контрольный в висок или затылок.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.857	❗️Чеченские бойцы успешно противостоят бандеровцам, нацикам и шайтанам на территории Украины.
	0.834	Флаг в Берлине, который установили Михаил Егоров, Мелитон Кантария и Алексей Берест, символизировал окончательное крушение нацизма.
	0.830	Или будет Франция. И я не про свежие круассаны и багеты по утрам.
	0.825	Редакция не будет фантазировать на тему прецедентов и возможности доставки в Румынию или Болгарию тральщиков с их дальнейшей передрапировкой на бандеровские флаги
	0.824	По данным Bloomberg, на следующей неделе Украина планирует национализировать Сенс-банк российских бизнесменов Михаила Фридмана и Петра Авена.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.143	Удар пришёлся по улице Университетская. Это оживленная центральная улица, на которой находится большое количество магазинов.
	9.057	Первые фото с места ликвидации боевиков, которые сбили на машине сотрудников полиции. В центре — «Мерседес», которым управляли боевики.
	9.027	Черноморский флот отработал задержание судна-нарушителя, а также произвел пуски противокорабельных крылатых ракет по мишени в северо-западной части моря. Ракетный катер «Ивановец» поразил цель, условный противник был уничтожен.
	8.973	Кстати, участие британцев тут нельзя исключать. Так как, в британской прессе нападению на Грайворон уделяют передовицы. Американцы пишут об этом намного скромнее.
	8.950	На улицах испанской Барселоны массовые демонстрации в поддержку Палестины. Кадры появились в соцсетях.
Input question: Украинские военные делают из мирных жителей живой щит.
Top-5 lexical search (BM25) hits
	29.550	Украинские военные подвергли опасности мирных жителей.
	24.325	При отступлении украинские военные начали обстреливать местность из «Градов», погибли двое мирных жителей. Об этом военкору Семёну
	22.951	 Украинские военные, добровольно сложившие оружие, рассказали о приказе «Правого сектора» расстрелять мирных жителей в Рубежном.
	21.799	Украинские коллаборанты грабят дома мирных жителей Донбасса
	20.530	На территории медицинского комплекса больницы №4  Мариуполя найдены тела мирных жителей и пациентов больницы. Это не массовое захоронение, это гора тел - жертвы нацистского режима, которые АЗОВ использовал как живой щит - прямое доказательство геноцида.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.829	Террористы из Азова удерживают в Мариуполе гражданских в качестве живого щита.
	0.827	Мирные жители города Херсона оказывают содействие российским военнослужащим в поиске военных преступников и националистов и желают только скорейшего налаживания мирной жизни.
	0.826	Украинские военные подвергли опасности мирных жителей.
	0.821	Один из таких жителей, по имени Валерий, рассказал, что его держали в подвале в Мариуполе. По его словам, националисты использовали людей как живой щит и никого не выпускали, несмотря на серьезные травмы у жителей города.
	0.817	Местные жители взламывали двери и витрины и растаскивали товары и вещи, технику.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.729	Те украинские (и не только украинские) командиры, которые ответственны за использование мариупольцев как «живого щита» - рано или поздно предстанут перед трибуналом.
	8.694	 Украинских военных в Мариуполе попытаются убедить сложить оружие, чтобы не пострадали мирные жители - Басурин
	8.677	Украинские артиллеристы не перестают терроризировать жителей Донецка, нанося удары по жилым кварталам.
	8.637	В результате удара уничтожено до пятнадцати человек живой силы противника, а так же один миномет.
	8.556	Жизнь, это же не красивые презентации для высоких руководителей. Презентацией невозможно наносить непоправимый урон противнику.
Input question: Украина нарушила Минские соглашения.
Top-5 lexical search (BM25) hits
	22.747	Зеленский изначально считал Минские соглашения «невыполнимыми».
	17.667	"ВСУ действуют наиболее нагло, откровенно «забивая» на Минские соглашения, «съедают» серую зону пядь за пядью.
	17.239	Также Порошенко прямо заявил, что Минские соглашения ему были нужны только для того, чтобы оттянуть время.
	16.832	• Россия исходила из необходимости выполнить минские соглашения, приложила много усилий, чтобы вынудить западные страны исполнять обязательства;
	16.072	От себя я бы спросил Макрона, если ДНР и ЛНР сепаратисты, почему же тогда Европейские страны подписали минские соглашения?

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.825	Лене Милашиной скорейшего выздоровления.
	0.823	ЭКСКЛЮЗИВ⚡️Взрыв в Донецке, прогремевший с утра, произошел в центре города на пересечении улиц 25 лет РККА и бульвара Пушкина на месте снесенного здания.
	0.822	Решения о действиях относительно Крыма, в том числе, при поиске путей мирного урегулирования, должна принимать Украина, а не США — госсекретарь США Энтони Блинкен
	0.821	• Не только Украина стала полигоном для испытания биологического оружия США — в прицеле американцев Грузия, Молдавия, ряд стран ШОС, заявил Патрушев.
	0.803	Мания величия и коррупция довели Украину до плачевного состояния, считает экс-глава МИД Польши.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.956	Цель такого соглашения - успокоить Киев и сигнализировать Москве, что безопасность Украины имеет критическое значение для США и их союзников.
	8.897	• Не только Украина стала полигоном для испытания биологического оружия США — в прицеле американцев Грузия, Молдавия, ряд стран ШОС, заявил Патрушев.
	8.801	Решения о действиях относительно Крыма, в том числе, при поиске путей мирного урегулирования, должна принимать Украина, а не США — госсекретарь США Энтони Блинкен
	8.784	Собственно Украина в нынешнем виде это во многом - об этом писал даже Кучма в книге "Украина не Россия" - результат большевистской коренизации.
	8.772	Одна украинская специалистка заявила, что Россия в гонке дронов уже обошла Украину.
Input question: На Украине создают биологическое оружие.
Top-5 lexical search (BM25) hits
	29.036	США создавали биологическое оружие на Украине.
	16.379	По не понятным причинам объект, где может разрабатываться в том числе биологическое оружие, охраняют всего 3 человека.
	15.667	США создают рабочую группу для борьбы с коррупцией на Украине
	13.692	На Украине начинают раздавать оружие заключенным. Пошла жара.
	13.224	Важно⚡️Будет ли Япония поставлять оружие Украине?

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.936	США создавали биологическое оружие на Украине.
	0.862	Позже задержали еще одного гражданина Украины. Он был соучастником.
	0.850	Скоро неизбежное утро".
	0.840	Результаты бесконтрольной массовой раздачи огнестрельного оружия на Украине.
	0.835	Украинские боевики сделали базу в жилом секторе, но были обнаружены и разгромлены

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.323	Сегодея с утра, в Карабахе сбили азербайджанский дрон-разведчик. Силы ПВО НКР уничтожили в субботу в 06:38 разведывательный беспилотник вооруженных сил Азербайджана.
	9.104	Я рискну предположить, что Джонсон не смог обеспечить нужный ход событий на Украине, отчего она становится обузой. Писали, что он еще поссорился с военной элитой и с частью разведки.  Для перезапуска бизнес-проекта "украина" нужен другой менеджер.
	9.069	Джо Байден наложит вето на законопроект о выделении средств на оказание помощи Израилю, если он будет согласован в Палате представителей Конгресса США в нынешнем виде, без упоминания Украины.
	9.065	Но нам тоже не стоит сидеть сложа руки, вполне возможно, что проблему можно решить уже сейчас, для этого просто нужно создать специализированного  бота.
	9.062	Украинским солдатам сейчас надо бы в очередной раз призадуматься, за кого и для чего они воюют.
Input question: НАТО хотело на напасть на Россию.
Top-5 lexical search (BM25) hits
	14.361	Польский генерал Роман Полко заявил, что обстрел территории Польши был сделано для того, чтобы развязать новую мировую войну или напасть на альянс НАТО.
	13.105	В Испании укронационалисты узнали адрес Шария и попытались напасть на него толпой.
	12.708	«Как НАТО должно сдерживать Россию Владимира Путина»: Bloomberg
	12.341	«Мы прибыли в Москву заверить в том, что не позволим никому использовать афганскую землю для того, чтобы напасть на Россию или соседние страны, - заявил ТАСС официальный представитель политофиса движения Мохаммад Сохаил Шахин.
	12.282	Пентагон, АНБ и киберцентр НАТО в Таллине организуют кибератаки под украинским флагом на Россию, заявил Патрушев.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.818	На Хоттабыча всем наплевать.
	0.800	Курс рубля ослабить до произвольной отметки можно ЗАВТРА. Вернуть бюджетной правило, например, только покупать на него юани, золото, рупии... Не хотите, нет?
	0.784	Хитрость удалась, враг потратил не только время, но и свои боеприпасы. Абсолютно в пустоту.
	0.780	Хоть ракеты с неба, но только не домой.
	0.777	Видео⚡️Как ВСУ начинали наступление на Работино

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.217	Россию они хотят новую построить, и власть кошмарную забороть. Списки составлять научитесь для начала.
	9.012	Хоп! И ничего преподавать уже не можешь. Ни на татарском, ни на эстонском не умеешь. Французским настолько не владеешь. Да и в европейских колледжах не устроиться.
	8.995	Хитрость удалась, враг потратил не только время, но и свои боеприпасы. Абсолютно в пустоту.
	8.894	И что характерно, никто хлопцев не бьет, не мучает, не пытает. В ноги не стреляет. Горло не режет.
	8.810	Хотя я и не готов в революцию. Нам бы поступательное развитие наладить.
Input question: В войне виноваты США, Запад и НАТО.
Top-5 lexical search (BM25) hits
	17.120	• США и коллективный Запад не стремятся к войне с Россией или третьей мировой войне, поставляя вооружения на Украину;
	13.139	The Hill: НАТО и США
	13.031	А когда Китай начнет бомбить Тайвань, США будут виноваты, или снова нет?
	12.893	При этом одновременно сообщает нам, что Запад един, как никогда, Германия решительна, как никогда, НАТО крепок и ждёт финнов и шведов, а США мировой лидер.
	12.731	В смерти Никиты виноваты не только украиские мясники-артиллеристы. Не только ублюдки из НАТО, которые накачивают Украину оружием.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.801	Сенсационное видео сирийского ребенка, записывающего себя, застрявшего под завалами, стало вирусным на Западе.
	0.797	В Гаити, между тем, растет недовольство США и Западом
	0.782	И не верить Западу.
	0.781	Трамп подал заявку на выдвижение в президенты США.
	0.780	Становитесь медиа сами: пишите тексты, записывайте видео, поддерживайтесь тех, кто уже этим занимается, не проходите мимо несправедливости и откровенного зла.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.396	Тут уж каждый сам решит - у нас закон зачастую несовершенен, или те, кто за законностью следят.  Не то чтобы прямо всё не здорово. В большинстве случаев всё как надо происходит. И садятся виновные в ДТП, невзирая на. Но запоминаются не большинство случаев, а те, где вопиющая несправедливость.
	9.254	Сенсационное видео сирийского ребенка, записывающего себя, застрявшего под завалами, стало вирусным на Западе.
	9.250	В общем, петицию у них на сайте ты заполняешь сам. Я предпочел Кембридж, все таки солидное заведение, с историей.
	9.191	Турки, Израиль, Азербайджан, а за спиной у них США. Ну то есть как "за спиной". Воевать каждый будет за свой интерес. Точнее будет так думать, а, в конечном счёте, за интересы США.
	9.169	Становитесь медиа сами: пишите тексты, записывайте видео, поддерживайтесь тех, кто уже этим занимается, не проходите мимо несправедливости и откровенного зла.
Input question: Санкции Запада против России доказывают, что они ненавидят Россию.
Top-5 lexical search (BM25) hits
	24.050	Сербии придётся ввести санкции против России из-за сильного давления Запада.
	20.202	Глава МИД Венгрии Петер Сийярто заявил, что санкции против России не работают, "они провалились"
	20.115	Немцы выступили против антироссийской позиции Запада и требуют отменить санкции.
	18.038	В США начинают подозревать, что Байден ввёл санкции не против России, а против своих граждан.
	17.948	Президент Кубы Мигель Диас-Канель Бермудес 22 ноября выступил перед депутатами Госдумы РФ, где осудил односторонние санкции Запада, которые несправедливо вводятся против России.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.868	Западная разведка не выявила причастности России к дестабилизации ситуации в Нигере —  Pais
	0.863	Не уверен, что заход на российскую территорию был запланирован. Тут или просто слегка заплутали и группа эвакуации и диверсанты.
	0.862	Те, кто постоянно завораживает любые проекты закона о репатриации.
	0.861	Празднование Дня России в Москве завершается традиционным салютом.
	0.857	Сербии придётся ввести санкции против России из-за сильного давления Запада.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.875	Мобилизованный, сражающийся на Донецком направлении, рассказал, как принял решение прийти в военкомат и отправиться в зону СВО.
	8.729	США заявили Турции, что «пора» принять две страны в альянс, а Литва заявила Анкаре, что задержка наносит ущерб региональной безопасности стран Балтии.
	8.656	Коалиция стран Запада, направленная против России, начала слабеть из-за экономических трудностей и опасений, что кризис на Украине затянется, сообщает The New York Times.
	8.626	Сербии придётся ввести санкции против России из-за сильного давления Запада.
	8.622	Ряд стран ЕС призвали Брюссель внести поправки в санкции против Москвы, чтобы они никак не мешали поставкам зерна и удобрений.
Input question: Если США можно устраивать войны на чужих территориях, то и России можно.
Top-5 lexical search (BM25) hits
	20.603	Но если очень хочется, то можно.
	20.285	Можно злиться и не соглашаться, можно ругать, уезжать, но разочароваться в России можно только если не любить её изначально.
	19.062	Можно стать героем войны.
	18.923	Если посмотреть на историю Владислава Лобаева, то можно увидеть его постоянную
	18.501	Если же будет решение бросить Украину в топку одну, по варианту "Грузия 2008" (если получат от русских, то и не жалко, а мы на Западе посмотрим, как русские действуют), то войны вполне можно ожидать даже до окончательной передачи власти в США.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.870	Если там останутся прежние персонажи, то, учитывая нынешние реалии, линия Ровшана и Джамшута может выглядеть так.
	0.839	Чехия передала киевскому режиму оружие, которым убивают наших мирных людей, наших детей.
	0.838	Но схема не так уж и важна. Куда важнее, чтобы у лиц, принимающих решения, было понимание, что ЧВК Вагнер нужно сохранить. Да  вероятно, трансформировать, изменить, но сохранить.
	0.835	Каждый ВСУшник может остаться в живых, если пойдет на условия российских военнослужащих и сдастся в плен.
	0.834	На слушаниях в Конгрессе США по вопросам абортов врач заявил, что мужчины тоже могут рожать.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.031	Если уж мы не стали угрожать в Латинской Америке, то здесь наши удары и ответы должны были быть ещё более сокрушительными для того, чтобы быть убедительными и сыграть свою роль в стратегическом сдерживании.
	8.552	Если там останутся прежние персонажи, то, учитывая нынешние реалии, линия Ровшана и Джамшута может выглядеть так.
	8.509	Все это можно и нужно было предотвратить, если бы каждый начальник принимающий  решения на своём уровне уже написал себе над своим рабочим местом.
	8.426	Танки Израиля уже вошли на территорию анклава, так что можно считать, что операция уже началась.
	8.393	Троих уже бывших членов приемной комиссии задержали, на них возбуждены уголовные дела по статье о мошенничестве. Их уволили первыми, позже расторгли контракты еще с шестью сотрудниками, которых разоблачили в содействии.
Input question: Запад ненавидит, не уважает и отменяет русскую культуру и всё русское.
Top-5 lexical search (BM25) hits
	21.938	• Уничтожить русскую культуру невозможно;
	21.060	Сказал что-то про русскую культуру и русский язык.
	18.310	"Ах, украинцы переименовали улицу Булгакова. Ах, они запрещают русский язык и русскую культуру".
	17.129	Может создатель акмеизма не считается крупным поэтом? Или есть мнение, что Гумилев никак не повлиял на русскую культуру?
	16.230	Однако, почему Васильев, человек, который формировал информационную повестку, снимался в фильмах, имел всё в России, всё же ненавидит "эту страну" ? Думаю, это какая-то болезнь. Ведь он искренне ненавидит, не за деньги.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.871	На заметку тем, кто поддерживает Украину и донатит ВСУ.
	0.853	• Запад курирует атаки БПЛА, которые спецслужбы Украины направляют на Россию, в том числе на Москву и Крым.
	0.853	Какие интересные сотрудники  у Умара Кремлева. Затейливые.
	0.851	Он также предложил распространить зону ответственности польской ПВО на Украину.
	0.849	Пусть вместе каются. За участие во вторжении в независимую страну, последующие беды иракского народа, за ИГ, за развал государственности и так далее.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.647	На Западе рассчитывают, что Путин – рациональный человек, который никогда не применит ядерное оружие.
	8.587	Наемникам тоже рекомендую ознакомиться. Всех вас, упыри, знают, вычислят, найдут, начинайте привыкать к земле.
	8.537	И нам стоит понимать, что быстро ничего не закончится. И впереди, скорее всего, зимнее наступление ВСУ. Уж кем они будут наступать, это проблема Киева. Но наступление будет. Как и усиление активности в части нанесения ударов по нашим тылам и аэродромам.
	8.537	Ну и заодно уж, надеюсь руководству Суперкубка стыдно за страну? Раз уж они решили соответствовать, то надо бы и постыдиться. И символику Z всенепременно запретить.
	8.441	А да, зарплаты не растут, как, впрочем, и у нас. И да, покупателтная способность евро тоже падает.
Input question: России не оставили выбора, кроме как начать военную операцию.
Top-5 lexical search (BM25) hits
	23.610	Шойгу: нам не оставили выбора, надо признавать
	18.752	В своем Рождественском послании робот Амека раскритиковала специальную военную операцию России на Украине.
	18.509	Срочно⚡️Азербайджан начал военную операцию в Арцахе
	17.500	Дело не в том, что она не поддерживает военную операцию. Имеет право.
	17.346	То есть по сути мне не оставили выбора. Именно поэтому я не ходил на выборы. Чтобы совесть моя была чиста.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.878	Спецрепортаж WG⚡️Курильские острова: Русский мир на Крайнем Востоке
	0.859	А то летают всякие Риперы возле нашего Крыма. Выполняют "обычные операции" по наведению украинских ракет и БПЛА на цели.
	0.856	• Освобождение всей территории Донбасса остается неизменной целью специальной-военной операции;
	0.854	Несколько капельниц и строгий медицинский контроль — огромное спасибо врачам.
	0.847	НАТО отклонила запрос Сербии на введение контингента полиции и армии в Косово — Вучич

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.755	Очевидно, что Киев репетирует на Донбассе «карабахский сценарий». Надеемся, командиры в республиках это осознают.
	8.755	Папа Римский Франциск пояснил, что слова о "великой России", за которые он подвергся критике со стороны представителей Украины, касались богатой культуры и исторического наследия страны.
	8.715	Нужны рации, коптеры? Есть интенданты. В Минобороны дверь всегда открыта. Постучались - получили. Но надо постучаться! А все эти закупки в Китае, проблемы на границе - все это непрофессионально (с) военный эксперт Алексей Леонков, 24.04.2022
	8.714	Не молчит она и на запорожском направлении. Обстрелы позиций ВСУ практически на всей линии соприкосновения. Цель – удары по концентрирующимся украинским войскам. Третье оперативное направление их наступления вполне себе вырисовывается.
	8.666	Нарастание количества непосредственных участников и стран, якобы принимавших участие в этой «операции», каждый с абсолютно противоречивыми подробностями, порождают обоснованные вопросы и сомнения в её реальности и, тем более, в успехе.
Input question: На Украине притесняют русских и русский язык, происходит геноцид русских.
Top-5 lexical search (BM25) hits
	22.763	Не любишь русских? Русский язык, культуру, не уважаешь традции, культуры и языки народов России? Ну хорошо.
	21.574	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
	21.574	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
	20.018	«Я русский и мне повезло» это новая песня-манифест. Для русских и про русских всех национальностей. Слушайте.
	19.323	Или когда вообще само слово "русский" выдавливали из лексикона, а русских на Украине и в Белоруссии заставляли учить национальные "языки"?

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.878	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
	0.878	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
	0.872	Российская граната прилетела в нескольких метрах от украинцев, повезло.
	0.871	В Грузии граждане Казахстана стирают оскорбительные надписи, оставленные украинцами в адрес русских.
	0.866	В сети расходится видео, на котором иноагент Максим Галкин произносит лозунг украинских националистов.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.786	Сегодня внимание россиян, по понятным причинам, приковано к войне на Украине, но из Арцаха, где развёрнут российский миротворческих контингент, поступают тревожные новости.
	8.733	Ни украинский «град», ни донбасская распутица, ничто не может остановить российских морпехов, штурмующих укрепрайоны украинских националистов.
	8.682	Украинские ублюдки, расстрелявшие российских военнослужащих, не должны понести никакого иного наказания, кроме того, что является чистой карой. Даже если на их поиски уйдут годы — Дмитрий Медведев
	8.643	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
	8.643	Русских на протяжении десятков лет притесняют в прибалтийских государствах, ущемляют в правах и дискриминируют по национальному и языковому принципу.
Input question: Украина - искусственная, ненастоящая страна.
Top-5 lexical search (BM25) hits
	11.072	В многосерийном фильме "Доставить любой ценой", страна производства Украина от 2011г., в эпизодической роли советского солдата-предателя, засветился
	9.500	Украина страна гладиатор. Которую готовят на убой. А вот население Украины, как рассказывают знакомые, у кого там родственники, совершенно очаровано скорой победой над москалями.
	9.422	Страна,
	9.149	Когда-то, мне один  проофессор, литературовед пытался объяснить, что "Василий Тёркин" это совершенно искусственная пропагандистская штука, потому что на войне людям не до шуток. А Твардовский, значит, специально преукрашивал реальность.
	9.077	ВСЯ СТРАНА

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.855	Украина:
	0.835	Украина
	0.835	Украина
	0.835	Украина
	0.835	 Украина

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.203	Украина - сконструированная система на пространстве цивизационного разлома (по Л.Н.Гумилеву - супер-этнического контакта, причём тройного: Россия, католический Запад, тюркский мир).
	9.111	Украина - это не просто не феншуйная страна, это талисман неудач, все к чему она прикасается и имеет отношение превращается в труху, ровно так, как называется популярный украинский ТГ канал.
	8.898	 Украина - не только самая большая страна Европы, но и с потенциалом, который не снился никому: грандиозное и очень разработанное сельское хозяйство, выход к морям, доставшаяся в наследство от СССР богатейшая промышленность,  недра, горы, культура невероятная, талантливый большой народ, святыни, намоленные многими веками, окно и дверь в разные европы, дружественные тебе соседи-братья: Москва и Минск...
	8.302	Как сообщают украинские каналы, столица украинского государства сейчас подвергается массированной ракетной атаке. На Киев пошла уже третья волна ракет.
	8.251	«Украина – агрессор»: жительница Сум не стала защищать режим и высказалась за правду
Input question: Президент Зеленский — наркоман, клоун и марионетка Запада.
Top-5 lexical search (BM25) hits
	17.839	А то что у Украины президент клоун? Тоже мне открытие
	16.432	• Президент Украины Зеленский встретится с президентом США Байденом — NBC
	16.015	• ПВО — наше слабое место — президент Украины Зеленский на Украинском завтраке в Давосе
	15.991	Президент Зеленский пытается убедить Германию поддержать поставку Украине боевых самолетов — AFP
	15.390	Клоун

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.874	Зеленский, не вынимая рук из карманов, заявил, что контроль над Мариуполем потерян. Навсегда.
	0.826	«Зеленский ведёт себя как капризная примадонна».
	0.824	Президент Украины Владимир Зеленский заявил, что Киев не может пока начать контрнаступление
	0.818	Мангистауская область на западе Казахстана. Нищие, безработные и доведённые режЫмом до отчаяния казахи на двух крузаках, лексусе и камри (плюс одна лада для народности) нападают на колонну Нацгвардии.
	0.814	• Политика Запада — кровавая и грязная, она отрицает суверенитет стран и народов.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.084	Коалиция стран Запада, направленная против России, начала слабеть из-за экономических трудностей и опасений, что кризис на Украине затянется, сообщает The New York Times.
	9.064	Зеленский, не вынимая рук из карманов, заявил, что контроль над Мариуполем потерян. Навсегда.
	8.945	Президент Украины Владимир Зеленский заявил, что Киев не может пока начать контрнаступление
	8.896	Зеленский внёс в Раду законопроект о праздновании Рождества на Украине 25 декабря — как на Западе.
	8.883	❗️ Путин подписал закон, предполагающий наказание до 12 лет колонии за незаконный экспорт из России или передачу иностранцам военных товаров, технологий или вооружений.
Input question: Президент Путин знает, что делает.
Top-5 lexical search (BM25) hits
	15.961	"У него определенно была другая встреча. Я не был разочарован, я был расстроен, потому что хотел бы с ним встретиться. Зеленский взрослый и знает, что делает", — бразильский президент.
	15.570	Похоже, все смешалось в доме Облонских, и правая рука не знает, что делает левая.
	15.082	 Президент РФ Владимир Путин
	14.785	Также выступая на ПМЭФ Путин отметил, что ничего не знает о том, как разворачивали самолет. Таким образом российский президент дал понять, что этот инцидент даже не заслуживает его внимания.
	14.683	У человека непосвященного вообще может возникнуть ощущение, что государство настолько рассредоточено, что одна структура не знает, что делает другая.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.795	Палестина-Израиль: что известно на данный момент
	0.795	Палестина-Израиль: что известно на данный момент
	0.795	Палестина-Израиль: что известно на данный момент
	0.795	Палестина-Израиль: что известно на данный момент
	0.795	Палестина-Израиль: что известно на данный момент

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.394	Президент Бразилии заявил журналистам, что «не знает, задержит ли Путина бразильское правосудие», если тот приедет в страну. Президент Бразилии отметил, что это решает судебная власть, а не правительство.
	9.343	Президент Перу Кастильо задержан, пишут местные издания. Полиция страны в своем заявлении назвала его экс-президентом.
	9.126	Президент ответил, что от избыточной наивности и доверчивости в отношении Запада.
	9.044	Многие про него просто не знают, другие забыли. Да и какая страна была тоже отчасти стерлось из памяти.
	9.044	Многие про него просто не знают, другие забыли. Да и какая страна была тоже отчасти стерлось из памяти.
Input question: О своих нельзя говорить плохо. Нельзя осуждать власть.
Top-5 lexical search (BM25) hits
	26.708	Осуждать его точно нельзя. И злословить тоже.
	24.194	Что можно, а что нельзя говорить о войне на Ближнем Востоке: ARD выпустило 44-страничную инструкцию для своих сотрудников
	23.097	"Нельзя не осуждать любые, а тем более агрессивные действия на почве национальной неприязни, которые не способствуют сохранению мира и межэтнического согласия". И вроде верно все сказано.  Нельзя не осуждать агрессивные действия.
	22.479	Итог. Пока ни о каком успехе нацистов на Харьковском фронте говорить нельзя.
	22.396	Потомку графа нельзя говорить "я — русский".

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.862	Самое глупое, что сейчас можно сделать — это вместо создания условий для их возвращения начать поливать их грязью.
	0.827	Теперь вот главное. Откуда взялась стоимость в два с половиной миллиона? Ну о которой писали в каналах.
	0.825	Главную улицу назвать в честь Вохи.
	0.818	Главное, не стоить иллюзии. Не возгордиться. Не подменять реляциями реальность.
	0.806	Мы разучились, знаете, и особо радоваться и слишком уж сильно горевать.Но главное на свободе.  В этой партии последние из моих друзей оказавшихся пять лет назад в тюрьме.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.412	Но обстоятельства так сложились. Против нашего нежелания не признавать или не думать. Господь не оставил нам вариантов. И у нас был один выход. Или вспомнить кто мы, где земля наших предков, что такое Новороссия, что мы русские, в конце-концов. Или исчезнуть.
	9.275	Позиция Лии Меджидовны может нравиться или нет. Но она имеет на неё полное право. Государство, в свою очередь, может не захотеть платить за эту позицию.
	9.097	Мы разучились, знаете, и особо радоваться и слишком уж сильно горевать.Но главное на свободе.  В этой партии последние из моих друзей оказавшихся пять лет назад в тюрьме.
	9.059	Вот еще подумалось про Навального, который набрасывает на Нюту и требует от нее «свалить в пользу Соболь». Из всей этой истории, выходит, что только он имеет право выбирать, кто из независимых кандидатов и где может выдвигаться. Мания величия на лицо.
	9.023	Президент Бразилии просто отказался встречаться и уехал, от греха.  Видно, чтобы вирус украинства не подхватить. Но это не так уж важно.
Input question: Путин – президент, которого выбрало большинство. Он представляет интересы большинства
Top-5 lexical search (BM25) hits
	21.496	Абсолютное большинство выбрало российского лидера. Меньше всего проголосовали за Байдена.
	17.691	Что из себя в итоге представляет большинство заявлений Навального
	17.638	Байден, которого он, Рейган, отправлял к Брежневу, президент страны.
	15.163	Он тоже прирожденный кшатрий, для которого война - кайф и развлечение. Особенно, если эта справедливая война за интересы своей Отчизны.
	15.082	 Президент РФ Владимир Путин

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.813	Только  политику делает не большинство. А пассионарное меньшинство. Вот оно сейчас и ведёт Армению в пропасть.
	0.809	Пентагон подтвердил, что ВС США сбили воздушный шар КНР по поручению Джо Байдена.
	0.806	Также Байден пообещал, что если Россия будет действовать вразрез с интересами США, то Вашингтон ответит. Кстати, Путину он предложил встретиться летом в Европе.
	0.805	ПыСы: кстати там сейчас в Инсте Вкусвилла беснуется агрессивное ультралевое меньшинство и прочие лгбт , предлагаю пройти туда и дать им отпор, защитив бренд, который уважает наше мнение.
	0.803	• Польша будет отстаивать свои интересы в вопросе импорта украинского зерна и продолжит блокировать его поставки после 15 сентября, заявил президент Анджей Дуда.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.731	Полный он будет после выборов,  на которых победит  Пашинян. А произойдёт это потому, что большинству граждан Армении плевать на то, что происходит. Было бы иначе - власть снесли бы после поражения в Карабахе.
	8.731	Полный он будет после выборов,  на которых победит  Пашинян. А произойдёт это потому, что большинству граждан Армении плевать на то, что происходит. Было бы иначе - власть снесли бы после поражения в Карабахе.
	8.253	Пентагон подтвердил, что ВС США сбили воздушный шар КНР по поручению Джо Байдена.
	8.168	Подход креативный. Может быть, благодаря этой акции, к которой, конечно, имеют отношения и военные, удастся восстановить полную картину произошедшего.
	8.088	Пишут, что подрыв авиабазы в Крыму - это не прилет, а все-таки диверсия.
Input question: Западные медиа и платформы социальных сетей агрессивно настроены по отношению к России и поддерживают дискриминацию русских.
Top-5 lexical search (BM25) hits
	19.554	🇷🇺 Россияне в социальных сетях активно поддерживают флешмоб
	17.951	Американский президент товарищ Дональд Трамп подписал указ, который  снимет с социальных сетей защиту от претензий по редактированию контента.
	17.670	Мощный взрыв в метро в Вашингтоне⚡️ Об инциденте сообщают американские пользователи твиттера и других социальных сетей.
	17.378	Немного смущает наше нездоровое высокомерие по отношению к врагам. Особенно – по отношению к Соединенным Штатам.
	16.814	— Также не допускается демонстрация российскими СМИ логотипа организации Meta Platforms Inc. и принадлежащих ей социальных сетей Facebook и Instagram.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.905	Мы наблюдали информационный обстрел со стороны западных СМИ и наших информационных власовецев, на протяжении почти двух месяцев.
	0.900	• Путин подписал закон о денонсации соглашения между РФ и Украиной по Азовскому морю и Керченскому проливу
	0.894	По предварительной информации, попадание снаряда пришлось в район сахарного завода, практически в центре населённого пункта.
	0.891	Запред Совфеда Андрей Турчак и глава ДНР Денис Пушилин приехали на мобилизационный пункт в прифронтовую зону в столице ДНР.
	0.887	По информации агентства, США предлагают союзникам запретить весь российский экспорт как минимум в определенных категориях с исключением для некоторых позиций.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	7.039	Спецоперация проходит не только на земле и в воздухе. Полномасштабные информационные боевые действия развёрнуты и по линии соприкосновения, и в глубоком тылу.
	6.881	На Запорожском направлении ВСУ после массированных артобстрелов закрепились в северной части села Работино и продвинулись в районе Вербового. ВС РФ наносили удары по украинским подразделениям в Степном (не путать с одноимёнными населёнными пунктами на Угледарском и Донецком направлениях), Пятихатках, Павловке, Малых Щербаках, Малой Токмачке, Красной Кринице и Чаровном. Артиллерия ВСУ била по российским позициям в Жеребянках, Нестерянке, Копанях, Новопрокоповке, Новопокровке и Новофёдоровке. (Рис. 1)
	6.876	Зафиксировано несколько попыток ротаций противника и проведения разведки наших передовых позиций малыми пехотными группами, которые были пресечены нашими войсками.
	6.822	Запред Совфеда Андрей Турчак и глава ДНР Денис Пушилин приехали на мобилизационный пункт в прифронтовую зону в столице ДНР.
	6.802	Заявление Минобороны РФ по поводу расправы украинских солдат над пленными российскими военными:
Input question: Модерация западных соцсетей продиктована политическими интересами.
Top-5 lexical search (BM25) hits
	12.430	Видео из соцсетей:
	11.610	Видео из украинских соцсетей: мужчину
	11.295	Катков Михаил Никифорович, 1863 год "Совпадение интересов украинофилов с польскими интересами".
	11.295	Что не должно быть пренебрежения интересами народа России в пользу инородцев.
	10.891	Кадры из соцсетей: утверждается, что на них

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.875	После таких заявлений дипломатическими договорённостями с немецкими канцлерами и французскими президентами можно подтереться.
	0.874	В администрации отметили, что Киев запросил у Вашингтона новую бронетехнику для подготовки контрнаступления.
	0.870	Почитал комментарии в Инстаграмме под постом от Вкусвилл, где компания признаёт, что слегка переборщила с модными западными тенденциями.
	0.866	Акция против конституционного закрепления размера МРОТ, индексации пенсий и создания системы сдержек и противовесов — это определенно митинг, который мы заслужили.
	0.860	Акция закончилась массовыми задержаниями.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.807	Моих сотрудников задерживали на протесных акциях, они в автозаки попадали, ну в горячке не поняли, что есть аккредитация. Разобрались, отпустили.
	8.766	Испанская полиция снова оцепила здание посольства Украины в Мадриде, сообщают местные СМИ.
	8.667	Мобилизованные смоляне отправились в зону проведения Специальной военной операции.
	8.641	Акция против конституционного закрепления размера МРОТ, индексации пенсий и создания системы сдержек и противовесов — это определенно митинг, который мы заслужили.
	8.603	Возле здания администрации президента Майи Санду в Кишинёве устроили протестную акцию со свечами.
Input question: Оппозиционные активисты ведут экстремистскую деятельность, направленную против России.
Top-5 lexical search (BM25) hits
	17.959	ПРЕМЬЕРА⚡️«Джихадисты» против России – пути ведут в Турцию
	15.235	Климатические активисты продолжают протест против угольной шахты в Лютцерате
	15.217	В нем, мы расскажем, как Запад и их прокси, ведут против России гибридную войну.
	15.084	Она снимала столкновения на границе Венесуэлы и Колумбии. По её словам, на неё наехал автомобиль, которым управляли оппозиционные активисты, — выжила чудом
	13.744	*Деятельность организаций запрещена на территории России

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.867	Депутаты обещают осенью на сессии комплексно рассмотреть вопрос. Невероятная скорость реакции.
	0.863	Людям не нравятся дегенеративные постсоветские элиты, а "оппозиционеры" это как правило вскормленные всякими западными НКО товарищи.
	0.863	О том, что карта национально-этнической преступности будет использоваться западными непартнерами для ослабления тыла России не написал только ленивый.
	0.863	Наступление и оборона  - две разные дисциплины военного искусства. Если обороняться украинцы более-менее умеют, то для наступательных действий требуется совсем другой уровень оперативного искусства.
	0.862	Безопасность движения колонн обеспечивают ударные вертолеты армейской авиации.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	7.830	Это всё, что вы должны знать о репутации автора колонки Медведева, опубликовавшем этот текст говноресурсе и я надеюсь что товарищ Симоньян, публикующая эту невежественную гнусность ответит если не должностью (о да, она неприкосновенна), то хотя бы репутацией.
	7.740	Англичане отправили из Индии экспедиционный корпус, который довольно быстро китайцев разгромил.  И заставил платить и каяться. И курить опиум.
	7.710	Людям не нравятся дегенеративные постсоветские элиты, а "оппозиционеры" это как правило вскормленные всякими западными НКО товарищи.
	7.705	В итоге Щёкинский эксперимент постепенно свернули, автоматизации от американцев отказали.
	7.691	Теперь вот одинокий французский военспец ушёл в горы навечно. Альпинизмом занимался индивидуальным.
Input question: Россия защищает своих граждан от дезинформации и внешних угроз, блокируя доступ к запрещённым сайтам.
Top-5 lexical search (BM25) hits
	18.901	— Национальная гвардия Техаса взяла под контроль участок границы США с Мексикой, блокируя доступ агентов федеральной пограничной службы
	17.778	Суть претензий военной прокуратуры к этим сайтам не уточняется.
	16.642	Австрия предостерегла своих граждан от поездок в российские города
	16.179	✔️Депутаты считают, что признание ДНР и ЛНР создаст основания для обеспечения гарантий безопасности и защиты жителей республик от внешних угроз, а также для укрепления международного мира и региональной стабильности.
	15.725	США скрывают от своих граждан информацию о состоянии ядерного вооружения страны

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.911	ВС РФ оснащены самым современным оружием и, вопреки многочисленным сообщениям Запада, далеки от деморализации.
	0.907	Экономическое сообщество стран Западной Африки закрывает границы между странами сообщества и Нигером, прекращает коммерческие рейсы — заявление организации.
	0.902	• Гражданам, подлежащим призыву, будет запрещено уезжать из РФ со дня размещения в отношении них повестки в реестре врученных повесток — поправки
	0.894	— Страны Персидского залива готовы оказать финансовую помощь Газе после завершения конфликта, но при условии существенных реформ в Палестинской национальной администрации.
	0.892	 Кадры уничтожения хранилища с бензином и дизельным топливом Кременчугского нефтеперерабатывающего завода, с которого осуществлялось снабжение  группировки ВСУ в центральных и восточных районах страны.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	7.705	Реакция киевского режима на разрушение гражданской инфраструктуры свидетельствует о его террористической сущности — официальный представитель МИД РФ Мария Захарова
	7.663	❗️Укронацистская инфопомойка "Ищи своих", созданная для террора мирного населения России и телефонного мошенничества, из-за недостатка контента начала постить трупы ВСУшников, выдавая их за российских военнослужащих.
	7.646	• Каждый гражданин в новых регионах РФ должен чувствовать себя под защитой закона, волнующие людей вопросы должны решаться в кратчайшие сроки.
	7.610	• ВС РФ в условиях поставок Запада оружия Киеву продолжают наращивать боевую мощь, в том числе за счет совершенствования подготовки;
	7.577	• Гражданам, подлежащим призыву, будет запрещено уезжать из РФ со дня размещения в отношении них повестки в реестре врученных повесток — поправки
Input question: Запад создаёт фейки для обвинения России в военных преступлениях.
Top-5 lexical search (BM25) hits
	24.402	• По подтвержденной несколькими источниками информации, Киев намерен осуществить масштабную провокацию для обвинения РФ в "военных преступлениях"
	22.617	• Киев использует любую возможность для обвинения России и попыток консолидировать Запад;
	20.896	Всем четверым предъявлены обвинения в преступлениях против человечества и в соучастии в военных преступлениях. В Дамаске информацию об ордере на арест в отношении Асада пока не комментировали.
	17.019	Экс-омбудсмен Украины призналась, что украинские власти вместе с разведкой просили ее раскручивать фейки о преступлениях ВС РФ.
	16.983	РФ передала в ООН данные о военных преступлениях Киева

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.874	• Турция выступает за продолжение выполнения условий зерновой сделки в четырехстороннем формате, сообщает диписточник ТАСС в Анкаре.
	0.874	А россиянам в республиках Донбасса предлагаю запомнить фамилии тех, кто выступает за лишение вас ключевых гражданских прав.
	0.860	• Киев использует любую возможность для обвинения России и попыток консолидировать Запад;
	0.858	Это какие финансово приятные взаимодействия с сотрудниками украинских и западных спецслужб.
	0.854	Россия укрепит военный потенциал на северо-западе в ответ на вступление Финляндии в НАТО, заявил замглавы МИД РФ Александр Грушко.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.810	Западная техника, активно используемая противником в начале контрнаступления на Запорожском фронте, в настоящее время всё реже появляется в зоне видимости. И происходит это потому, что с самых первых дней она стала желанной целью для наших бойцов.
	8.676	Финляндия не стала исполнять рекомендацию ЕС, которая запрещает российским туристам въезд на зарегистрированных в РФ автомобилях, пишут местные СМИ со ссылкой на МИД Финляндии.
	8.639	Ввод западных истребителей позволит применять на линии фронта полный спектр западного авиационного вооружения. По задумке JDAMы и HARMы будут лететь точнее, российские войска дрогнут, и вот тогда в бой снова массово кинут западную бронетехнику.
	8.630	Россия укрепит военный потенциал на северо-западе в ответ на вступление Финляндии в НАТО, заявил замглавы МИД РФ Александр Грушко.
	8.624	Киев может активизировать попытки наступления в ближайшие недели из-за опасений, что приход зимы и осадки осложнят ситуацию на фронте, пишет швейцарское издание Temps со ссылкой на украинских военных.
Input question: Украина создаёт фейки для обвинения России в военных преступлениях.
Top-5 lexical search (BM25) hits
	24.402	• По подтвержденной несколькими источниками информации, Киев намерен осуществить масштабную провокацию для обвинения РФ в "военных преступлениях"
	20.896	Всем четверым предъявлены обвинения в преступлениях против человечества и в соучастии в военных преступлениях. В Дамаске информацию об ордере на арест в отношении Асада пока не комментировали.
	17.019	Экс-омбудсмен Украины призналась, что украинские власти вместе с разведкой просили ее раскручивать фейки о преступлениях ВС РФ.
	16.983	РФ передала в ООН данные о военных преступлениях Киева
	16.126	Французское информационное агентство AFP, ссылаясь на свои источники, утверждают, что в рамках дела о применении химического оружия всего было выдано четыре ордера на арест. Кроме Башара Асада, обвинения также предъявлены его брату Махеру Асаду, командиру Республиканской гвардии Сирии (элитного военного подразделения, предназначенного для защиты сирийских властей), а также двум генералам, имена которых не разглашаются. Всем четырем лицам предъявлены обвинения в преступлениях против человечества и содействии в военных преступлениях.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.879	Главное из заявления СК о преступлениях украинских формирований:
	0.875	Видео прислано бойцами с Бахмутского направления, но вероятно оно актуально и для других участков фронта.
	0.871	Фриковатый Даня Милохин, очевидно малоприятный для меня персонаж. Но вся ситуация вокруг него еще более неприятная и неправильная.
	0.869	Очередные доказательства военных преступлений украинских вооруженных формирований.
	0.865	На донецком фронте российская армия атакует в районе Новомихайловки, от Старомихайловки пытается продвинуться к укрепрайону ВСУ в Красногоровке.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.809	Зенитная ракетная система "Фаворит" предназначена для поражения современных и перспективных самолетов, стратегических крылатых ракет, тактических и оперативно-тактических баллистических ракет и других средств воздушного нападения.
	8.779	 Для уничтожения укреплений применяются снаряды на фугасное действие, что позволяет нанести максимальный урон фортификационным сооружениям, в том числе заглубленным бетонированным укреплениям противника.
	8.633	Около 3 часов утра украинские формирования силами до двух мотопехотных рот, усиленных танками, предприняли попытку вторжения. Самоотверженными действиями российских военных отражены три атаки украинских террористических формирований.
	8.622	Фриковатый Даня Милохин, очевидно малоприятный для меня персонаж. Но вся ситуация вокруг него еще более неприятная и неправильная.
	8.614	Активизация ВСУ на линии соприкосновения - переброска войск к фронту из центральной Украины - заявления Киева про какие-то новые договорённости по Донбассу - обещания, что 15 марта будут изменения в ситуации - фейк про российских военных в Донецке.
Input question: Цель западной пропаганды - дегуманизировать русских.
Top-5 lexical search (BM25) hits
	17.799	Вот что интересно, так это постоянство западной пропаганды.
	13.379	Все, носящие оружие украинцы, - самая благодатная среда для западной пропаганды. Теперь ими больше занимаются не местные, а западные спецы.
	11.553	Очень верно пишет Андрей Медведев о неизменности западной пропаганды. Потому что мы и нужны Западу как вечная и неизменная Россия, со всем набором придуманных там штампов.
	11.333	Профессор университета из Торонто (индус по происхождению) задает очень неудобные для западной пропаганды вопросы. Украинцы уже требуют выгнать его из учебного заведения и принести им публичные извинения.
	10.632	На земле и в небе русская армия громит русских же зомби, с промытыми за 30 лет пропаганды мозгами. При условиях тотальной машины военного-психологического террора, работающей на Запад и выращивающей из русских детей нацистских зомби.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.848	При этом, площади расположенного рядом торгового центра использовались в качестве крупной базы для хранения реактивных боеприпасов и перезарядки систем залпового огня.
	0.846	Запад целенаправленно спровоцировал конфликт на Украине для сдерживания России — Путин.
	0.843	В одном из реабилитационных центров Мариуполя сегодня отменили занятия, а воспитанников выгнали копать окопы.
	0.842	• Запад прилагает руки к деятельности террористов в Афганистане;
	0.841	• России удалось компенсировать закрытие западных рынков для нее;

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.850	Запад целенаправленно спровоцировал конфликт на Украине для сдерживания России — Путин.
	8.840	Ранее текст законопроекта был опубликован на сайте Верховной рады. По словам одного из депутатов, его цель — «предотвращение угроз национальной безопасности Украины и обеспечение порядка».
	8.835	По делу Цюмана, Муханкина, под подозрение попадали и задерживались невиновные люди.
	8.810	Ради нашего выживания, нам просто необходимо запретить пропаганду ЛГБТ и трансгендерности целиком и полностью.
	8.748	В Казани полицейские задержали спекулянта сахаром. Барыга пытался продать 50-килограммовый мешок по цене значительно выше рыночной, но покупателями оказались силовики.
Input question: В России дружественная медиасреда, плюрализм мнений, и нет цензуры,
Top-5 lexical search (BM25) hits
	14.771	Крайне рекомендую к прочтению. Внятно и четко изложение как работает информационная сеть,  точно не дружественная России.
	13.814	Зачистка Дебальцево⚡️Исходники без цензуры
	12.829	▪️Сербия — дружественная страна, а Белград "принципиально против антироссийских санкций"
	12.829	Россия не закрывает - видимо, Британия для нас самая дружественная страна )
	12.042	Удивительно, как быстро и управляемо готовят европейцев к переговорам. Это же не отдельные всплески мнений, нет, это буквально хором поют все СМИ.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.828	Президент России В.В.Путин: "Я надеюсь, что до таких провокаций дело не дойдет, а если это случится, мне думается, что это будет иметь очень тяжелые последствия для украинской государственности в целом".
	0.826	Тут даже вопрос не денег, а целеполагания и смыслов, стратегии продвижения национальных интересов России.
	0.826	В Средней Азии, как мы и прогнозировали давным давно, ситуация перманентно накаляется.
	0.825	Я уже писал про то, как добрые русские люди финансируют изделия на целые подразделения. Снайперские взводы,  роты... Их имена для помянников - Дмитрий и Виктор.
	0.825	Я уже писал про то, как добрые русские люди финансируют изделия на целые подразделения. Снайперские взводы,  роты... Их имена для помянников - Дмитрий и Виктор.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.859	Президент России В.В.Путин: "Я надеюсь, что до таких провокаций дело не дойдет, а если это случится, мне думается, что это будет иметь очень тяжелые последствия для украинской государственности в целом".
	8.825	Радио России и Сретенская духовная академия запускают цикл лекций, ну или если угодно, бесед на самые разные темы.
	8.650	Мы порой ворчим, рассуждаем, что России не хватает дерзости в действиях на международном направлении.
	8.593	- Да все, как всегда, - констатируют мужики. – Связи между разными соединениями толком нет. Что у соседей происходит, никто понять не может. Пока допросишься работы артиллерии по нашим целям через многоступенчатую систему докладов, целей уже и след простыл.
	8.592	Мы в нашей истории строим царство, разделившееся в себе. Которое никогда не будет сильным и надёжным.
Input question: Украинские военные не заботятся о благополучии украинских граждан
Top-5 lexical search (BM25) hits
	15.791	• РФ весьма негативно оценивает решение европейских стран о поставке тяжелой техники на Украину, они не заботятся о будущем украинцев.
	15.475	Путин подписал указ о призыве на военные сборы граждан, пребывающих в запасе
	15.243	Граждан Украины только жаль. Американские компании заработают. Украинские военные умрут. Немного неравноценная сделка, да. Но что же делать.
	14.795	Украинские военные устроили
	14.340	В начале апреля этого года в Мариуполе украинские военные задержали двух мирных граждан, проезжавших на велосипеде мимо блокпоста. Гражданские лица представились мужем и женой. Несмотря на наличие у них украинских паспортов, в которых стояла отметка о прописке в Мариуполе, их заперли в подвале.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.852	Глава ГУР МО Украины Буданов, отвечая на вопрос о причастности украинских спецслужб к убийствам известных российских деятелей, заявил:
	0.850	Пекин не предлагал оставить за Россией захваченные территории Украины — глава МИД Украины Кулеба
	0.850	🎖 Полковнику Алексею Бернгарду присвоено звание Героя России за выполнение боевых задач во время военной операции на Украине.
	0.845	 Задержан украинский военнослужащий, причастный к убийству и сексуальному насилию в отношении мирных граждан.
	0.842	Глава МИД Великобритании Джеймс Клеверли: Законное место Украины в НАТО.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.583	Украинский истребитель Су-27 гоняется за российской крылатой ракетой в небе над Одессой сегодня утром.
	8.537	Затрагивая Украину в контексте грядущей встречи с Байденом, российский лидер отметил, что возникла необходимость более широко высказаться по теме истоков украинской государственности, а также о событиях Второй Мировой войны, связанных с этим регионом.
	8.482	Глава украинской военной разведки говорит, что у России достаточно высокоточных ракет для нанесения еще нескольких массированных ударов, прежде чем у нее закончатся запасы — Reuters
	8.459	Заместитель начальника ГУР Минобороны Украины Вадим Скибицкий заявил о сложной ситуации для украинских военных на линии боевого соприкосновения и попросил больше оружия
	8.457	Но, учитывая, как умело наш противник, а точнее НАТО и западные гражданские политтехнологи, работают с психикой граждан Украины, рискну предположить  такой вариант.
Input question: Украинские военные терроризирут мирных жителей Донбасса, ДНР и ЛНР.
Top-5 lexical search (BM25) hits
	29.550	Украинские военные подвергли опасности мирных жителей.
	29.523	Украинские коллаборанты грабят дома мирных жителей Донбасса
	23.066	Силы ЛНР начали эвакуацию мирных жителей Северодонецка
	22.951	 Украинские военные, добровольно сложившие оружие, рассказали о приказе «Правого сектора» расстрелять мирных жителей в Рубежном.
	22.723	Украинские боевики заперли мирных жителей в подвале и заминировали — бойцы ЛНР спасли семью из украинского плена

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.844	Украинские коллаборанты грабят дома мирных жителей Донбасса
	0.822	А мне вот скорее интересно другое. Как они там все будут жизнь устраивать это их личное дело.
	0.815	Мрия так и осталась несбывшейся украинской мечтой. Такой же далёкой и недоступной, как Крым, ДНР, ЛНР...
	0.810	Местные жители взламывали двери и витрины и растаскивали товары и вещи, технику.
	0.810	Русские люди, которые живут на Донбассе, должны жить в России.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.525	Украинские неонацисты не прекращают наносить беспорядочные удары по частным домам, в которых живут мирные люди.
	8.391	Вот смотрите. Американцы бьют с дрона по машине, в которой, якобы, террорист. То ли был, то ли нет. Убивают четырех детей. Я вообще удивлен, что потери такие. Узкая же улица, дома кругом.
	8.298	И вот теперь одни украинские военные преступники бьют по другим. А вот эти другие, понимают наконец, как жил Донбасс все восемь лет. И возможно, понимают, что это вот они превращали жизнь мирных людей в ад.
	8.292	День за днём украинские террористы уничтожают гражданскую инфраструктуру Донецка и дома местных жителей.
	8.185	Лубок скорописный, како же мериканский змей летучий да ворожбе ученый, рекомый "Жнец", со тверди небесной низвержен бысть.
Input question: Запад контролирует украинские СМИ.
Top-5 lexical search (BM25) hits
	15.177	Украинские СМИ
	14.141	Тем временем, украинские сми
	13.674	Украинские СМИ опубликовали видео как
	12.827	Украинские СМИ публикуют кадры из Угледара (ДНР).
	12.827	Украинские СМИ публикуют фото Киева во тьме.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.867	Какие интересные сотрудники  у Умара Кремлева. Затейливые.
	0.862	• Россия воюет не столько с украинской армией, сколько с коллективным Западом;
	0.854	Отдельное спасибо за урок.
	0.850	Видимо, я упустил момент, когда в России запретили советскую символику, как в Прибалтике и на Украине
	0.846	Уважаемый Константин, спасибо Вам огромное за отклик!

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.301	Сначала украинские военные разбомбили их дом, а затем, отступая, заперли всех в подвале, установив несколько растяжек.
	9.243	Зеленский уволил заместителя командующего Нацгвардии Украины Руслана Дзюбу, соответствующий указ обнародован на сайте ОП
	9.215	Зачем украинскому полицейскому УК и УПК, когда в руках есть дубина, а в мозгах вакуум. Застал на месте преступления и тут же привёл приговор в исполнение.
	9.143	• Запад бросает украинцев в топку. Украина прошла путь прямого, неприкрытого вмешательства западных стран в ее внутренние дела.
	9.083	КНДР запустила неопознанную баллистическую ракету в сторону Японского моря, заявили в Сеуле - РИА Новости
Input question: Запад хочет контролировать пользователей рунета и доступ россиян к интернету.
Top-5 lexical search (BM25) hits
	21.375	По мнению Бернса, китайское правительство может запросить у соцсети доступ к информации американских пользователей.
	17.760	Издание отмечает, что НАТО — последняя организация, которая запретила соцсеть из-за опасений, что китайское правительство может получить доступ к данным пользователей TikTok.
	15.783	Поэтому не удивляйтесь, что как только появляется доступ к интернету, солдаты будут читать вражеские ресурсы. А люди гражданские будут искать информацию точно там же. Других источников у них просто нет.
	14.971	ХАМАС сообщил агентству AFP, что в секторе Газа «по большей части» отсутствует доступ к интернету. Подтверждение проблем с интернетом получено от независимой организации NetBlocks, специализирующейся на мониторинге доступа в сеть по всему миру.
	14.902	ЕС хочет сделать въезд россиян в союз фактически невозможным, отметили в постпредстве.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.855	Коммунисты в очередной раз - коллективно - поднимают вопрос о зарплате для всех депутатов.
	0.847	КНДР запустила баллистическую ракету в сторону Японского моря, власти Японии попросили жителей острова Хоккайдо уйти в укрытия из-за опасений падения ракеты рядом с островом.
	0.839	КНДР запустила неопознанную баллистическую ракету в сторону Японского моря, заявили в Сеуле - РИА Новости
	0.835	Местный депутат Ростислав Антонов написал заявление в прокуратуру.
	0.834	Что хочется отметить. У замминистра есть доступ ко всем базам данных  населения полуострова.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.414	Я знаю, что за КПП, знаю даже кто из сотрудников забрал коптеры. Но не хочу писать этого. Пока.
	8.230	И хорошо, что Геннадий Андреевич дал совет украинцам, как им поступать. А то они же не понимали, запутались совсем, не знали что делать. Теперь пойдут выберут себе нормального президента.
	8.181	Снова хочу задать вопрос: кто такие представители диаспор? Что это такое диаспора? Субъект власти? ОРД? Где, в каком законе Российской Федерации написано, что чиновники что-то должны согласовывать с диаспорами?
	8.157	Путин на заседании Совбеза поручил принять дополнительные меры по контрразведке, выявлению и розыску террористов
	8.136	Другая часть светских консерваторов хочет ограничить влияние аятолл и настроена на расширение диалога с Западом.
Input question: Украина совершает военные преступления против своих граждан.
Top-5 lexical search (BM25) hits
	30.359	2. Украина использует террористические методы ведения войны и регулярно совершает военные преступления.
	18.994	А что там в Европах, введут санкции против Киева за военные преступления?
	18.237	ВОЕННЫЕ ПРЕСТУПЛЕНИЯ УКРАИНЫ
	17.733	В США начинают подозревать, что Байден ввёл санкции не против России, а против своих граждан.
	16.504	Очень важно, чтобы украинские власти и военные уже сегодня знали - все их преступления против мирного населения Донбасса, а также все их преступления против жителей на территориях, в настоящий время подконтрольных Киеву - детально фиксируются.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.903	Дискусия тоже неплохо прошла. Сразу становится ясно: государственные каналы всё врут, всех покрывают, правду не говорят.
	0.873	Все задержанные, входившие в раскрытую группу украинских диверсантов - дали признательные показания.
	0.867	Пострадавших среди гражданского населения и сотрудников правоохранительных органов нет.
	0.867	Реакция главы Забайкалья Александра Осипова на происшествие с ветеранами СВО вызывает глубокое уважение.
	0.863	Госдеп: США осуждают применение силы российскими правоохранителями к участникам несогласованных акций протеста.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.515	Гутерриш также подчеркнул, что ситуация в анклаве ужасная, не хватает продовольствия, воды, лекарств.
	8.494	Против войны на Украине и против власти Путина готов заниматься мошенничеством. Таким образом организаторы украинских преступных групп вербуют российских мошенников для совершения преступлений.
	8.392	Выживший при взрыве мариупольского театра мужчина рассказывает о преступлениях украинских боевиков.
	8.360	Реакция главы Забайкалья Александра Осипова на происшествие с ветеранами СВО вызывает глубокое уважение.
	8.265	Для семей погибших участников СВО государством предусмотрена ежемесячная денежная компенсация.
Input question: Запад вмешивается во внутреннюю политику России.
Top-5 lexical search (BM25) hits
	13.709	• Запад всячески пытается вмешаться в ситуацию в Нагорном Карабахе и дискредитировать российскую миротворческую политику;
	13.411	Запад полностью контролировал внутреннюю политику страны, попытка поменять ситуацию местной элитой, уйти из под контроля,  закончилась первым в истории переворотом - Майданом,  инициированным ЦРУ. Это история премьера Моссадыка, почитайте.  Во внешней политике Иран был постностью проамериканским, хотя и финансово контролировался из Лондона. Оппозиция была полностью уничтожена, а душила ее спецслужба САВАК – ее создали американцы и беглые гестаповцы.
	13.411	Запад полностью контролировал внутреннюю политику страны, попытка поменять ситуацию местной элитой, уйти из под контроля,  закончилась первым в истории переворотом - Майданом,  инициированным ЦРУ. Это история премьера Моссадыка, почитайте.  Во внешней политике Иран был постностью проамериканским, хотя и финансово контролировался из Лондона. Оппозиция была полностью уничтожена, а душила ее спецслужба САВАК – ее создали американцы и беглые гестаповцы.
	12.903	Честно, у меня ответа нет. При этом я убежден, что сама война вовсе не была ошибкой. Ошибкой было влезть в сложную  афганскую внутреннюю политику, не понимая её.
	11.777	Заместитель руководителя АП Сергей Кириенко, отвечающий за внутреннюю политику - отправился в административную столицу будущего российского региона - для того, чтобы запустить там важный с точки зрения формирования новой региональный элиты конкурс «Лидеры Возрождения».

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.891	Татарский правильным вопросом задаётся:
	0.874	СРОЧНО⚡️Генштаб ВС Армении выступил за отставку Пашиняна
	0.865	• Запад всячески пытается вмешаться в ситуацию в Нагорном Карабахе и дискредитировать российскую миротворческую политику;
	0.863	❗️При зачистке Мариуполя российские силовики задвухсотили всушного снайпера.
	0.851	Вообще не только Запад не понял Россию и русский народ, о чем верно пишет Владлен. Прошедшие девять месяцев разрушили массу иллюзорных представлений о России, внутри неё самой.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.013	Запад не решается конфисковать российские активы, потому что опасается, что это может «подорвать веру в доллар».
	8.914	Заявление вполне логичное. Для Британии, с её весьма системным подходом к внешней политике, нынешняя ситуация это шанс поквитаться за Крымскую ещё войну, и новая попытка отрезать России от незамерзающих морей. Чем, собственно, Британия занимается со времени русского покорения Крыма.
	8.834	• Запад всячески пытается вмешаться в ситуацию в Нагорном Карабахе и дискредитировать российскую миротворческую политику;
	8.783	• Запад спекулирует на нуждах пострадавших от землетрясения в Сирии, продавливая в СБ ООН расширение механизма трансграничной помощи.
	8.764	Вообще не только Запад не понял Россию и русский народ, о чем верно пишет Владлен. Прошедшие девять месяцев разрушили массу иллюзорных представлений о России, внутри неё самой.
Input question: Западные СМИ ведут информационную войну против России.
Top-5 lexical search (BM25) hits
	21.216	В нем, мы расскажем, как Запад и их прокси, ведут против России гибридную войну.
	20.528	«Есть способ прекратить войну на Украине»: Западные СМИ все дальше
	18.517	Зачем Запад ведет против России информационную войну? Когда и почему это война началась? И почему после 24 февраля эта война приобрела характер нездоровой истерии?
	17.959	ПРЕМЬЕРА⚡️«Джихадисты» против России – пути ведут в Турцию
	17.908	Западные СМИ

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.916	Заодно включив и нацистский слоган в официальную переписку.
	0.906	По предварительной информации, попали в авиационный завод.
	0.905	Трофейное западное оружие, захваченное российскими военными на позициях ВСУ на Изюмском направлении.
	0.895	Любые недружественные акции со стороны Запада получат своевременный и адекватный ответ, заявили в МИД РФ.
	0.889	• Пахотными землями Украины владеют западные корпорации, именно они бенефициары инициативы.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.778	С западной стороны Работино войскам РФ удалось выбить украинцев с захваченных ими ранее позиций, вернув под свой контроль очередную лесополосу и восстановив линию фронта.
	8.720	Зеленский заявил о дефиците вооружений и боеприпасов у ВСУ и добавил, что страны Запада должны ускорить их поставки.
	8.710	Любые недружественные акции со стороны Запада получат своевременный и адекватный ответ, заявили в МИД РФ.
	8.698	5️⃣ Русские запускают программу стимулирования рождаемости, многодетных семей с выплатой вознаграждений и другие национальные проекты модернизации инфраструктуры (включая новый космодром);
	8.680	На Запорожском направлении ВС РФ при поддержке артиллерии давят на ВСУ на западных окраинах села Работино. Украинские штурмовые подразделения ведут аналогичные действия к северо-западу от Вербового. (Рис. 2)
Input question: Зеленский хочет уволить или уволил Залужного из-за ревности к его популярности.
Top-5 lexical search (BM25) hits
	24.522	-хочет популярности ?
	22.771	Паниковский просит Джанго Освобождённого, уволить Залужного. Холст, масло, кокаин.
	18.405	Зеленский хочет
	18.101	Зеленский уволил Григория Галагана с должности командующего Сил специальных операций ВСУ и назначил на его место Виктора Хоренко.
	17.918	Зеленский объявил, что решил уволить Алексея Резникова с поста министра обороны Украины, и предложил на его место Рустема Умерова.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.858	Зеленский заявил, что ВСУ сложно наступать из-за дальнобойного оружия России.
	0.829	Известно, что ранее задержанный уже привлекался к уголовной ответственности за хулиганство.
	0.827	Не может. Вчера приняли нацистский закон о языке. Первый шаг к полному запрету русского на Украине.
	0.825	Срочно⚡️Путин подписал указы о признании Херсонской и Запорожской области независимыми территориями
	0.824	— Участники каирского саммита по Палестине не примут итоговое заявление из-за разногласий между арабскими и европейскими делегациями.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	7.971	• Заокеанские хозяева киевского режима не позволяют сдвинуть ситуацию вокруг Украины с мёртвой точки.
	7.953	Известно, что ранее задержанный уже привлекался к уголовной ответственности за хулиганство.
	7.846	Шведский публицист Ларс Берн завил, что НАТО добровольно утилизирует свое вооружение, отправляя его на Украину. Военный альянс при этом рискует остаться безоружным.
	7.820	Зеленский заявил, что ВСУ сложно наступать из-за дальнобойного оружия России.
	7.778	Задержан в аэропорту Внуково при попытке сбежать из России. Италию ушлепок увидит очень нескоро.
Input question: Залужный — следующий президент Украины.
Top-5 lexical search (BM25) hits
	14.470	• Президент Украины Зеленский встретится с президентом США Байденом — NBC
	14.248	• ПВО — наше слабое место — президент Украины Зеленский на Украинском завтраке в Давосе
	13.346	Залужный
	13.283	Залужный настаивал на мобилизации для II группы инвалидов — нардеп Безуглая
	13.234	🇺🇦⚡️Депутат Верховной рады Евгений Шевченко утверждает, что главнокомандующий Вооруженными силами Украины (ВСУ) Валерий Залужный

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.837	Затем, запретить денежные переводы из России в Таджикистан.
	0.834	Залужный, викликай Волгу і будь розсудливий.
	0.828	— рыжий маленький задира,
	0.820	Задержан за шпионаж в пользу Украины.
	0.815	Залужный

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.982	 "Задержанный" вчера сотрудниками ФСБ полковник Басурин передаёт привет Главному управлению разведки Минобороны Украины.
	8.938	Злоумышленники задержаны и им избрана мера пресечения заключение под стражу. Расследование продолжается.
	8.902	Приметы мужчины: 178 см, русые волосы, большой нос и зеленые глаза, среднее телосложение, морщинистое лицо, узкие брови, редкие зубы.
	8.896	«Украинцы, складывайте оружие. Заканчивайте, возвращайтесь домой», — призвал он.
	8.843	Задержан инженер, который вместе с живущим в Британии подельником организовал интернет-магазин.
Input question: Залужный исчез из СМИ из-за ревности и тревоги Зеленского.
Top-5 lexical search (BM25) hits
	16.019	Главком ВСУ Залужный, конечно все понимает. Смотрит на эту карту и на портрет Зеленского.
	13.493	По его словам, сначала офис Зеленского хочет добиться гарантий того, что Залужный не будет выдвигать свою кандидатуру на возможных выборах.
	13.346	Залужный
	13.132	Генпрокуратура России намерена дать оценку заявлениям Зеленского, цитируемым в российских СМИ.
	12.980	60-летний страх Карибского кризиса исчез, испарился, отомщён.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.817	Из-за того, что навигации не было, речники сидели без работы, а всю грузовую логистику перевели на наземные маршруты.
	0.814	P.s нам стесняться нечего, реквизиты не замазываем.
	0.802	Разногласия между Зеленским и Залужным
	0.797	• Нигер закрыл воздушное пространство из-за угрозы военной интервенции.
	0.790	• Из-за того, что на территории Украины идет активная война, невозможно "позвать Киев в альянс прямо сегодня или завтра"

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.757	Из-за того, что навигации не было, речники сидели без работы, а всю грузовую логистику перевели на наземные маршруты.
	8.701	А все от безнаказанности. Потому что раз за разом им сходит это с рук. Космонавтам они своей легкой рукой «перечеркивают заслуги», ветеранов называют предателями, заслуженным врачам, спасшим сотни детей, хамят, как провинциальные билетерши. При этом сами в жизни ничего созидательного не сделали и ничем героическим не отметились.
	8.633	Сражение за логистику: развернулись бои за один из главных маршрутов снабжения российской группировки в районе Изюма.
	8.604	Злые русские засели в Администрации президента и сеют рознь а также плодят ненависть.
	8.553	Маленькие магазины в Польше массово закрываются из-за экономической ситуации в стране.
Input question: Зеленский истеричный и слабый.
Top-5 lexical search (BM25) hits
	10.331	Сильный это сильный всегда и во всем. И если ты не сильный, значит слабый. Значит, можно с тобой поступать, как со слабым. Это не хорошо, не плохо. Так устроен регион. Да, что регион. Вся мировая политика так устроена. Слабый? На, получи.
	9.620	Ранее советник главы Офиса президента Украины заявил, что Индия и Китай имеют слабый интеллектуальный потенциал и не анализируют последствия своих действий.
	9.138	Зеленский
	9.138	Зеленский
	9.138	Зеленский

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.830	Волонтёры и сотрудники МЧС Армении разгребают завалы.
	0.818	«СРОЧНЫЕ НОВОСТИ: Зеленский
	0.804	Русские интересы имеют значение.
	0.791	В российских зонах нередки случаи, когда ислам принимают скинхеды-язычники, отбывающие срок за преступления ненависти.
	0.790	И украинские политики требуют запретить сериал, как инструмент российской мягкой силы.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.382	Зеленский, прибывший на саммит Европейского политического сообщества в Молдавии, встретился с Санду.
	9.379	В российских зонах нередки случаи, когда ислам принимают скинхеды-язычники, отбывающие срок за преступления ненависти.
	9.379	Страны НАТО истощили свои запасы артиллерийских снарядов для поставок на Украину, заявил генсек альянса Йенс Столтенберг.
	9.331	Ответственность за случившееся Зеленский возложил на российскую сторону, по его словам, удар был нанесен артиллерией.
	9.260	Лавров заявил, что Украина - потерянный случай с точки зрения выполнения Минских соглашений.
Input question: В Украине семья погибшего бойца не получила выплат от государства.
Top-5 lexical search (BM25) hits
	23.728	Семья Байдена получила $1 млн от
	16.296	Ребенок не уходит от погибшего отца.
	14.965	Имущество России в странах G7 останется замороженным до выплат Украине — СМИ
	14.697	Кроме страховой суммы в размере 3 миллионов семья погибшего участника СВО может получить единовременное пособие свыше 4 миллионов рублей.
	14.149	Видео⚡️Везение спасло бойца от верной смерти

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.860	Соглашусь полностью с тем, что написал Александр Коц. Скрывать имена погибших героев, практика глупая и порочная. Она в конечном счете обезличивает победу, подвиг, и размывает его.
	0.853	Своими повадками нацист очень напоминает Сашку Белого. Тот тоже любил и на камеру поиграть, и прокурорских потрепать за галстук.
	0.852	Сотрудники бара, рядом с которым погиб боец ГРУ, знали нападавших
	0.850	Вместо подавшего в отставку Плотницкого его место займет глава МГБ Леонид Пасечник
	0.849	 Вечером выпивали и гуляли по Москве. Саша Рогаткин, конечно, большой молодец. Он сам героически работал на Донбассе. Теперь вот такой крутой проект.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.732	В последние месяцы на окраинах Горловки идут нешуточные бои. Противник всеми силами старается пробиться в город, ведь в этом случае украинцам удастся одержать громкую победу в медиапространстве.
	8.683	Своими повадками нацист очень напоминает Сашку Белого. Тот тоже любил и на камеру поиграть, и прокурорских потрепать за галстук.
	8.664	Греция не будет передавать зенитно-ракетные комплексы С-300 на Украину, несмотря на просьбы США, так как это ослабило бы ее оборону, сообщил министр национальной обороны Николаос Панайотопулос в интервью греческому общественному телевидению ERT.
	8.620	В Одесской области на границе с Молдавией задержали очередного военнообязанного, пытавшегося покинуть Украину, переодевшись в женское платье.
	8.599	Взрыв прогремел на туристической улице Стамбула Истикляль. По предварительным данным, в результате происшествия в центре  города есть пострадавшие.
Input question: Зеленский хочет вести войну до последнего украинца.
Top-5 lexical search (BM25) hits
	27.314	Так что, до последнего украинца.
	27.076	• США подталкивают Украину вести войну до последнего украинца и не жалеют на это никаких средств, но это не может повлиять на ход спецоперации;
	26.441	Так и будут, до последнего украинца.
	23.445	Это очень яркая иллюстрация к фразе "война до последнего украинца".
	22.515	Другими словами, раз Пентагон сказал: "до последнего украинца" - значит, до последнего. Что в результате будет с Украиной, шерифа не волнует.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.850	Российские военнослужащие патрулируют улицы Херсона.
	0.846	Традиционно переводим с дипломатического на военно-политический:
	0.842	Зеленский уволил Григория Галагана с должности командующего Сил специальных операций ВСУ и назначил на его место Виктора Хоренко.
	0.838	Сегодня утром ВСУ нанесли очередные артиллерийские удары по Донецку. В нескольких районах города пропало электричество. Скорее всего, повреждены энергетические коммуникации.
	0.837	В Херсонской области полицейские нашли и уничтожили крупный тайник для диверсий.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.700	Вероятнее всего - они задержаны полицией. И вероятнее всего - задержание носит именно политический характер.
	8.683	 Вчера, в процессе зачистки населенного пункта Почепин, российский танк уничтожил в одном из домов ДРГ укронацистов.
	8.660	На Херсонском направлении ударом высокоточного оружия уничтожен пункт временной дислокации ВСУ с личным составом.
	8.651	Вот в принципе всё, что нужно знать об украинской политике и о состоянии страны на сегодняшний день.
	8.629	На самом деле или государство примет меры для защиты детей от убийц, педофилов-рецидивистов, и прочего биомусора, или это все закончится не очень хорошо.
Input question: Попытки Украины удержать Бахмут — это пиар на крови.
Top-5 lexical search (BM25) hits
	16.930	❗️Неудачный пиар экс-президента Украины Петра Порошенко на одном из блокпостов.
	16.197	• Битва за Бахмут — один из примеров отличной защитной стратегии Украины.
	15.410	В Артёмовск (Бахмут) переброшен батальон «Айдар», чтобы удержать позиции ВСУ и не допустить бегства мобилизованных с поля боя.
	14.342	🇷🇺 Создаётся устойчивое впечатление, что чинуши средней руки, отвечающие за пиар в разного рода ведомствах, — однояйцевые близнецы. Более того, работа строится по принципу «все так делают». Изучение рынка, попытки оседлать повестку, использование новых приёмов и площадок — это не про них.
	14.106	Министерство обороны сообщило о контроле над Артемовским, оно же Хромово. Во время битвы за город Артемовск (Бахмут) противник смог удержать этот село.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.854	Всё это присутствует на кадрах борьбы укровояк с российским БпЛА-камикадзе.
	0.851	 Эксклюзивные кадры. Рядовой Поросенко сумел выжить и покинуть Бахмут.
	0.847	Эта мера поддержки будет распространяться в том числе на усыновленных детей.
	0.843	Депутат Бундестага Петр Бистрон - о поставках танков на Украину.
	0.840	Украинские нацисты продолжают перенимать опыт боевиков-исламистов.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.802	По данным украинских властей, это был удар баллистической ракеты. По некоторым данным, в театре проходила выставка дронов.
	8.799	Просто было бы нам логично, в ответ на продажу "Байрактаров", отсыпать оружия и денег курдам. Миномётов и противотанковых установок побольше. Намекнуть, что теракт в Крыму готовили протурецкие боевики.
	8.770	Как уже было написано не раз, это обычная тактика бандитов и террористов. Хватать невинных людей, чтобы менять их на своих подельников.
	8.736	Московский Кредитный Банк блокирует переводы на карту вдовы, и те деньги, что уже поступили блокирует тоже.
	8.702	Бспилотники у России будут. Их сделают потом. Вот так распоряжаются бюджетными деньгами.  Посадить бы в окоп всех этих приближенных менеджеров, чтоб у них производственный педикулез побыстрее начался.
Input question: Украинская власть претесняет церковь.
Top-5 lexical search (BM25) hits
	18.276	УКРАИНСКАЯ ЦЕРКОВЬ ОТДЕЛИЛАСЬ ОТ РУССКОЙ?
	16.060	Украинская власть убивает русских детей.
	16.060	Украинская власть убивает русских детей.
	15.065	А нападать на Донбасс украинская власть побоится.
	13.582	«Очень прошу нашу власть прекратить гонения на Украинскую православную церковь — единственную каноническую православную церковь Украины, признанную всеми без исключения православными церквями мира», — говорит один из бойцов.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.846	УКРАИНСКАЯ ЦЕРКОВЬ ОТДЕЛИЛАСЬ ОТ РУССКОЙ?
	0.835	На криворожском участке украинские войска пытались продвинуться с севера в сторону Суханова. Цель – прорваться к Бериславу и Новой Каховке.
	0.835	• Стратегическая цель Украины вернуть все утраченные территории не выглядит достижимой военным путем в краткосрочной перспективе.
	0.822	Смерть нацизму⚡️Неизвестные активисты разместили провокационный банер на входе в украинский культурный центр в Москве⚡️
	0.818	Вот и все ципсошные закладки проявились. Ясно какие сетки телеграмм каналов управляются укронациками.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.427	Украинская прокуратура вызвала на допрос основателя ЧВК «Вагнер» Евгения Пригожина, его подозревают в «посягательстве на территориальную целостность страны».
	9.414	Украинские неонацисты не перестают обстреливать кассетами центр Донецка. Обстрелы проводятся как днём, так и ночью. Никакое военной цели они не преследуют. Удары наносятся по жилым кварталам Киевского и Ворошиловского районов.
	9.386	Украинские силовики вместе с представителями ПЦУ ворвались на территорию Преображенского собора Белой Церкви (Киевская область).
	9.346	На криворожском участке украинские войска пытались продвинуться с севера в сторону Суханова. Цель – прорваться к Бериславу и Новой Каховке.
	9.239	Как сообщает Центральное телевидение Китая, в Пекине прошёл семинар по национальной обороне и военной реформе с участием членов Политбюро ЦК Коммунистической партии Китая, и Си Цзиньпин направил участникам семинара ряд указаний.
Input question: Утечка секретных документов из Украины.
Top-5 lexical search (BM25) hits
	28.119	Утечка секретных документов по контрнаступлению ВСУ не должна подорвать отношения Украины со странами Запада, заявил советник главы офиса президента Украины Михаил Подоляк.
	27.648	Американские издания ранее писали, что утечка секретных документов Пентагона началась как раз с Discord.
	18.963	Утечка секретных документов американской разведки нанесла некоторый ущерб отношениям США с ключевыми союзниками, но значительного влияния на них не оказала, в отличии от разоблачений десятилетней давности экс-сотрудника Агентства нацбезопасности (АНБ) Эдварда Сноудена.
	18.065	В соцсетях появилась еще одна партия секретных документов США. Материалы касаются Украины, Китая и Ближнего Востока, сообщили источники The New York Times.
	16.559	Минюст США возбудил дело по факту утечки десятков секретных документов Пентагона о конфликте на Украине,

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.857	Об учебниках украинского языка.
	0.855	 Установлены как минимум два участника
	0.846	Таким образом, уже сейчас ведется подготовка для них украинских расчетов
	0.844	 Бастрыкин поручил СКР срочно возбудить уголовное дело в отношении украинского медика,
	0.838	Инструктор одного из учебных центров рассказал проекту

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.090	Очень красивые кадры из Артёмовска (Бахмута), на которых артиллерия "музыкантов" разносит в пыль укрепрайоны украинских боевиков.
	9.080	Венедиктов Леонид Никифорович имеет удостоверение участника боевых действий и отличительный знак «Участник АТО».
	9.050	Британские ученые начнут изучать эффективность созданных ими "умных носков" для людей, страдающих деменцией, сообщил телеканал Sky News.
	9.002	Дмитриев ублюдок и извращенец, который сломал жизнь ребёнку. Точка. Без всяких оговорок и скидок.
	8.984	Ракетным ударом вертолетов армейской авиации уничтожена бронетехника и живая сила противника.
Input question: кровавые преступления киевского режима
Top-5 lexical search (BM25) hits
	19.695	О боевиках киевского режима:
	19.045	 Несмотря на попытки киевского режима
	18.436	Капитуляция киевского режима - это правильная картинка.
	17.866	Одесские партизаны призвали бороться против киевского режима
	17.329	Ещё одно преступление киевского режима на земле Донбасса.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.839	Кадры атаки местных повстанцев на силы безопасности Мьянмы с применением автоматического оружия и РПГ.
	0.837	Ого! Вот это я понимаю «кровавый режим». Лидеров Каталонии приговорили к срокам до 13 лет тюрьмы из-за референдума. Бесславный конец попытки самоопределения.
	0.829	Криворожское направление: ожесточённые сражения в районе Высокополья.
	0.827	А для понимания, кому мы предлагали Рождественское перемирие.
	0.826	❗️ Движение автотранспорта по Крымскому мосту восстановлено в реверсивном режиме по крайней правой полосе направления Тамань – Керчь.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.153	По ее словам, "агония киевского режима, тупая ненависть, злоба и отсутствие какой-либо перспективы в собственном развитии и привели к этой террористической деятельности".
	9.150	Киевский режим, несмотря на поставки новых видов натовского вооружения, терпит поражение, заявил Шойгу.
	9.086	Кадры атаки местных повстанцев на силы безопасности Мьянмы с применением автоматического оружия и РПГ.
	9.075	Ого! Вот это я понимаю «кровавый режим». Лидеров Каталонии приговорили к срокам до 13 лет тюрьмы из-за референдума. Бесславный конец попытки самоопределения.
	9.016	Премьер-министр Великобритании Риши Сунак на Мюнхенской конференции по безопасности призвал удвоить поставки оружия Киеву:
Input question: радикально настроенные оппозиционеры
Top-5 lexical search (BM25) hits
	12.912	Прогрессивные сирийские оппозиционеры?
	12.471	Вот, скажем, отечественные оппозиционеры.
	11.313	Ну оппозиционеры традиционно  проявляют заботу о народе.
	11.020	Так что даже несколько десятков ликвидированных талибов радикально ситуацию не изменят.
	10.724	2. Никакая отдельно взятая боевая единица не способна радикально изменить соотношение сил.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.830	2) Нельзя верить «оппозиции», которая плевала на национальные интересы страны.
	0.820	Специальный корреспондент проекта
	0.804	Специальное включение корреспондента
	0.804	Оппозиционеры-экологи в Татарстане оставили после себя срач и горы одноразовой посуды
	0.800	Ополченец в Ямполе рассказал проекту

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.979	Нужны рации, коптеры? Есть интенданты. В Минобороны дверь всегда открыта. Постучались - получили. Но надо постучаться! А все эти закупки в Китае, проблемы на границе - все это непрофессионально (с) военный эксперт Алексей Леонков, 24.04.2022
	8.963	В одном из реабилитационных центров Мариуполя сегодня отменили занятия, а воспитанников выгнали копать окопы.
	8.867	Сокращенная процедура вступления Украины в НАТО — потенциально очень опасна для европейской безопасности — Песков.
	8.861	И да, напомню, что наполеоновская Франция перестала воевать, когда у Наполеона закончились мобилизационные ресурсы.
	8.826	Подтопление в дельте Нила может вызвать отрицательный рост экономики Египетского региона.
Input question: столкновения вооруженных радикалов с органами правопорядка
Top-5 lexical search (BM25) hits
	13.111	Правоохранительными органами Татарстана
	12.574	Во Франции продолжаются столкновения демонстрантов с полицией.
	12.245	Правоохранительными органами Кировской области пресечена
	12.245	Сейчас он задержан правоохранительными органами.
	12.196	В Ливане вспыхнули ожесточённые столкновения протестующих с военными.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.872	Эксклюзивные кадры поражения бронетехники противника на направлении Орехов-Работино.
	0.865	Военнослужащие комендатуры Красного Лимана и полиция МВД ДНР совместно организовывают мероприятия по поиску диверсионных групп противника.
	0.864	- От тропиков до полярных морей американская армия всех сильней
	0.862	В результате проверки правоохранителями установлена переписка в соцсетях, в которой задержанный указывал место положение населённых пунктов Каменско-Днепровского района, а также места для нанесения ударов.
	0.857	Приверженцы коммунистической идеологии Бердянска провели в центре города митинг в честь Дня рождения основателя СССР Владимира Ленина.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.077	Минутка доминирования. Российский танк ворвался на позиции укров и, сопровождая свое появление огнем по противнику, обратил вражескую пехоту в паническое бегство.
	8.993	Вызывают обеспокоенность действия украинских правоохранительных органов. По словам адвоката задержанной, Татьяну Кузьмич принуждают к самооговору, а на ее окружение оказывается давление с целью получения ложных показаний.
	8.815	С самого утра понедельника в образовательные учреждения республики поступают сообщения о минировании.
	8.809	Картинка с детьми (некоторых не гнушались измазать кровью), стариками и домашними животными в подвалах и метро, тысяча ракурсов одного повреждения. Откровенные постановки с попыткой выдать своих граждан за пленных русских, призывающихся одуматься
	8.798	Но, к сожалению, постановка «Современника» — это  лишь очередной шажок по глобальному перекраиванию отечественной культуры под глобалистский запрос. Если бы только это был единичный случай — идёт глумление над всем привычным традиционализмом во всевозможных отраслях.
Input question: 22 февраля 2014 года на Украине произошел насильственный захват власти или госпереворот.
Top-5 lexical search (BM25) hits
	28.763	Видео из Мариуполя 2014 года, время «Евромайдана». Так начинался захват власти и геноцид русскоговорящего населения на Украине.
	25.592	"Известия" 22 февраля 1992 года.
	20.323	💥 В ночь на 22 февраля 2014 года активисты Евромайдана заняли правительственный квартал, который покинули силовики. Вскоре они выдвинули множество требований, главное из которых — отставка президента Януковича.
	18.766	То, что происходит в Украине с 2014 года,
	18.453	• К спецоперации России на Украине привел устроенный там госпереворот.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.867	Реакция главы Забайкалья Александра Осипова на происшествие с ветеранами СВО вызывает глубокое уважение.
	0.858	Главное из заявлений Владимира Путина на встрече с выпускниками Высшей школы государственного управления РАНХиГС:
	0.854	Глава украинского министерства иностранных дел Дмитрий Кулеба заявил, что Украина не согласна на гарантии безопасности как альтернативу вступления в НАТО.
	0.850	На заседании совета глав государств СНГ Владимир Путин в очередной раз высказался о необходимости создания независимого Палестинского государства.
	0.850	— Исламское государство взяло на себя ответственность за плохо сделанные уроки;

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.603	В ДНР начинается мобилизация. Главный герой старается не выходить из дома и запоем читает новости. Вдруг он услышал тот самый звонок в дверь… Приближалось 22 февраля.
	6.082	Государственный переворот 2014 года был нарушением международного права, которое запрещает любое вмешательство во внутренние дела других государств.
	5.764	16 августа 2021 года президентским указом №465 полковнику Виктору Ивановичу Полосухину будет присвоено звание Героя России посмертно.
	4.908	Прозорливо увидел наш Патриарх, сказав недавно "Глянцевая идеалогия закончилась".
	4.747	Активная фаза наводнения в Херсонской области прошла, заявил врио главы региона Сальдо.
Input question: Во время «Евромайдана» оппозиция неоднократно применяла огнестрельное оружие и «коктейли Молотова», что привело к десяткам человеческих жертв.
Top-5 lexical search (BM25) hits
	21.826	Молчание всего мира во время войны в Арцахе привело к возрождению нацизма.
	21.117	По данным местных СМИ, демонстранты пытались создать блокпост возле АЗС. В силовиков кидали камни и «коктейли Молотова», в ответ был применён слезоточивый газ.
	17.418	Тут товарищ, представляющийся ветераном АТО, служивший в ССО ВСУ, рассказывает, как Украина применяла химическое оружие на Донбассе.
	17.418	Тут товарищ, представляющийся ветераном АТО, служивший в ССО ВСУ, рассказывает, как Украина применяла химическое оружие на Донбассе.
	16.824	Видео из Мариуполя 2014 года, время «Евромайдана». Так начинался захват власти и геноцид русскоговорящего населения на Украине.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.848	 Бойцы ОСН "Ермак" из зоны проведения специальной военной операции поздравляют с праздником ветеранов МВД.
	0.846	Девочка явно травмирована некими жизненными обстоятельствами. Судя по её интервью, есть и в образовании пробелы. И вообще ей,  судя по всему, мало кто толком уделял внимание.
	0.843	Добавляя к этому стремительно ухудшающуюся ситуацию с продовольствием - удобрения, посевная - можно прикинуть, в насколько интересные времена мы живём. Китайцы орут "Европа, одумайтесь, вы нас веками кормили образом мудрого белого человека, у которого всё просчитано и схвачено, мы за это деньги платили, чтобы ощутить сопричастность к великой белой науке "как всё по уму сделать" - что происходит-то с вами, ау?!!!!" - а их никто не слышит. Все заняты - памятники Пушкину перекрашивают и буквы латинского алфавита запрещают.
	0.838	Если "скудоумные руководители" НАТО примут Украину в альянс, Киев не откажется от попыток вернуть территории, России придется жестко отвечать всеми возможными средствами, заявил Медведев.
	0.834	Единственные позитивные персонажи сериала – русские атлеты Баширов и Петров, которые, приехав, чтобы купить спортивного питания и полюбоваться знаменитыми мостами, случайно становятся свидетелями сатанинского культа, во главе которого стоит перебежчик Скрипаль, приносящий котов в жертву королеве Елизавете (дочери Сатаны, как помним из первой серии). Как вы видите, я проделал гигантскую работу в архивах, отсмотрел сотни исторических документов, тысячи метров кинохроники, чтобы создать удивительно точный в деталях, практически документальный и поразительно реалистичный сериал о Великобритании и ее королевской семье.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.130	Как всегда, власть оказалась "единственным европейцем", и Собянин решился разрубить все эти клубки противоречий, которые накрутили вокруг Лубянской площади ребята из Общественной палаты и Клабхауса. Я все эти дни писал, что это голосование абсурдное. И его отмену нельзя не приветствовать. Памятники не должны разъединять. А еще, памятники должны соответствовать эпохе. Мы живем в эпоху научных открытий, изменения уклада мира, в эпоху гиперзвука в самом широком смысле этого слова. И России есть что в эту эпоху сказать. А не доставать из сундука старые образы, да еще и сея раздор. В общем, выдохнули.
	6.124	То есть в перспективе у Китая есть возможность сделать транспортный коридор от своих границ до Европы. Инфраструктурный мегапроект. И за счет него притянуть к себе союзников, партнеров, создать миллионы рабочих мест там, где они жизненно необходимы. И стабильный Афганистан, подконтрольный талибам, ровно тут кусок паззла, которого не хватало. География это судьба. Нет другой дороги с востока на запад. Со времен Шелкового пути.
	5.998	В прошлые выходные в Варшаве на улице Новый Свят озверевшая пьяная толпа укропитеков жестоко убила местного поляка, посмевшего заступиться за польских девушек, к которым приставали новоиспеченные "европейцы". Местная общественность разгневана молчанием польских СМИ о национальной принадлежности виновников  преступления.
	5.827	Но все приходит с опытом. Евреи тоже в конце сороковых и начале пятидесятых криво работали. Потом научились. Тенденция такова, что нужно усиливать украинское направление. И не жить воспоминаниями о 2014 в Дебальцево.
	5.771	Но есть одно важное отличие.  На Донбассе был не обычный риск.  На Украине была объявлена прямая охота на наших журналистов. Причем, началось это еще на Майдане. Когда только зародившийся Правый сектор усиленно искал где в Киеве живут "москальские пропагандисты". В список "Миротворца" журналисты попали практически первыми. Журналисты стали целями
Input question: В 2014 году в юговосточных областях Украины начались массовые акции против госпереворота.
Top-5 lexical search (BM25) hits
	24.265	Причина: "публичное отрицание российской агрессии против Украины в 2014 году".
	23.990	Напомним, протестные акции против позиции мониторинговой миссии начались после
	22.720	СБУ, первое из силовых структур было отмобилизовано новыми властями после госпереворота в 2014 году.
	20.694	В этот день, 7 апреля 2014 года, в ходе массовых протестов против госпереворота в Киеве, в пределах территории Донецкой области Украины, была провозглашена Донецкая Народная Республика!
	17.411	В центре Тбилиси снова начались столкновения между спецназом и участниками акции протеста.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.883	Ситуация на Донбассе, с вероятной подготовкой Украины к началу активных боевых действий, весьма и весьма напоминает события 2008 года. А именно попытку Грузии решить югоосетинскую проблему одним ударом.
	0.883	Ситуация на Донбассе, с вероятной подготовкой Украины к началу активных боевых действий, весьма и весьма напоминает события 2008 года. А именно попытку Грузии решить югоосетинскую проблему одним ударом.
	0.883	По поводу юридических гарантий нераспространения НАТО на восток. Вот красноречивая иллюстрация того, что Вашингтон - настоящий хозяин своего слова.
	0.875	Бахмут основал в 1571 году русский царь Иван Грозный на южном фронтире молодого русского государства. Бахмутская сторожа называлась крепостица. Потом вокруг выросла слобода. Основательную крепость постоили при при Петре Первом.
	0.874	Юрий Романов, один из первых советских еще стрингеров, отчаянный военкор,  войну в Карабахе 92-го года описывал так:

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.264	В 2001 году, во время гражданской войны в Македонии, местные офицеры-македонцы говорили, что в конечном счёте, задача шиптаров создать единое государство на обломках Югославии.
	5.881	В юбилейный год Достоевского (вообще чудо, что он состоялся, учитывая как забыли годовщину убийства Гумилева), перечитываем Фёдора Михайловича.
	5.697	Гипотетический арест в ЮАР президента РФ Владимира Путина по ордеру Международного уголовного суда (МУС) был бы равносилен объявлению войны России, заявил президент ЮАР Сирил Рамапоса в документах, направленных в Высокий суд южноафриканской провинции Гаутенг.
	5.658	Главы всех стран Африки приглашены на предстоящий в августе в южноафриканском Йоханнесбурге саммит БРИКС (Бразилия, Россия, Индия, Китай, ЮАР) для участия во встречах на его полях.
	5.652	Бобанич ликвидирован группой российских спецназовцев в пяти километрах южнее города Изюм. Лидер Львовской неонацистской группировки вступил в ряды «Правого сектора» в ноябре 2013 года. С 2014 г. участвовал в карательной операции киевского режима на Донбассе. Лично отдавал приказы на артобстрелы жилых кварталов Донецка и Луганска из тяжелого вооружения. Виновен в сотнях смертей мирных жителей ДНР и ЛНР, в том числе детей.
Input question: Крым провел референдум о вхождении в состав России в полном соответствии с международным правом.
Top-5 lexical search (BM25) hits
	23.272	НАТО осуждает выборы в новых регионах России, считает их "нарушением международного права", несмотря на то, что эти регионы вошли в состав России по результатам референдумов в соответствии с международным правом и в рамках признанного ООН принципа права на самоопределение, заявил официальный представитель альянса Дилан Уайт.
	21.897	ОРБ «Спарта» салютует в честь подписания договора о вхождении Донбасса в состав России💥
	20.825	В России приступили к подготовке командиров кораблей и экипажей к более РЕШИТЕЛЬНЫМ действиям в соответствии с международным правом на случай повторного нарушения иностранными военными кораблями/судами российской границы (тервод) после вторжения американского эсминца в территориальные воды России в заливе Петра Великого.
	20.702	В период проведения референдума о вхождении ДНР в состав России, Союзные силы активно используют наиболее мощные артиллерийские системы для подавления украинских боевиков.
	20.686	Глава ЛНР Пасечник заявил, что обязательно обратится к Путину с просьбой как можно быстрее принять решение о вхождении республики в состав России по итогам референдума

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.895	Министр иностранных дел РФ Сергей Лавров не видит перспектив для переговоров между Москвой и Киевом.
	0.893	• РФ поддерживает решение об оснащении миротворческих сил ОДКБ современными вооружениями
	0.892	В период проведения референдума о вхождении ДНР в состав России, Союзные силы активно используют наиболее мощные артиллерийские системы для подавления украинских боевиков.
	0.891	Мы уважаем религиозные чувства представителей всех конфессий, но зомбированные хохлы веселят даже дальнобоев.
	0.890	Пригожин прокомментировал фотографию с Милоновым в камуфляже, которую тот вчера разместил у себя в соцсетях.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.025	В период проведения референдума о вхождении ДНР в состав России, Союзные силы активно используют наиболее мощные артиллерийские системы для подавления украинских боевиков.
	7.989	В разгар обсуждений о деталях проведения референдумов на освобождённых территориях, противник не оставляет попыток забрать город, который вот-вот должен войти в состав РФ. Речь о Красном Лимане.
	7.785	По сообщениям военкоров, ВС РФ уже заняли первые дома и ведут ожесточённые бои в черте города. Инициатива на стороне российской армии.
	7.742	Посол Армении вызван в МИД РФ, ему выражено жесткое представление на фоне недружественных действий Еревана, сообщает ведомство.
	7.736	РФ денонсирует договор об обычных вооруженных силах в Европе, Путин назначил Рябкова своим представителем при рассмотрении парламентом — документ
Input question: В 2014 году власти Украины начали военную операцию против жителей юго-востока Украины, недовольных госпереворотом.
Top-5 lexical search (BM25) hits
	28.787	Причина: "публичное отрицание российской агрессии против Украины в 2014 году".
	23.930	С 2014 года у жителей Юго-востока и так изымают имущество.
	23.578	"Осудить военную агрессию РФ против Украины и аннексию Крыма".
	21.657	Самозванные власти Украины с 2014 года пичкали Мариуполь неонацистскими боевиками и оружием.
	19.397	В течение первой недели после крушения лайнера США пытались организовать военную операцию НАТО на востоке Украины. Под благовидным предлогом спасения обломков самолета и останков тел. Власти США, Голландии, Австралии и НАТО инициировали план

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.869	Бобанич ликвидирован группой российских спецназовцев в пяти километрах южнее города Изюм. Лидер Львовской неонацистской группировки вступил в ряды «Правого сектора» в ноябре 2013 года. С 2014 г. участвовал в карательной операции киевского режима на Донбассе. Лично отдавал приказы на артобстрелы жилых кварталов Донецка и Луганска из тяжелого вооружения. Виновен в сотнях смертей мирных жителей ДНР и ЛНР, в том числе детей.
	0.868	В частности юноша призывает всех участвовать в незаконных уличных акциях. А также рассказывает, как использовал на улицах Москвы опыт, полученный, видимо на тренингах за границах.  В итоге не странно, что люди с хмурыми и серьезными лицами обратили внимание на Жукова.
	0.866	"К югу от Бобринской бушевала, гикала, грохотала на бешеных тачанках, открывая с ходу пулеметный огонь, свистела, грабила, насиловала женщин и драпала при первой же встрече с сильным противником украинская черная вольница.
	0.865	В 2001 году, во время гражданской войны в Македонии, местные офицеры-македонцы говорили, что в конечном счёте, задача шиптаров создать единое государство на обломках Югославии.
	0.863	Украинские войска сосредоточили силы у границы с Приднестровьем, заявила пресс-секретарь сил обороны Украины "Юг" Наталия Гуменюк.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.247	В 1999 году войска НАТО убили несколько сотен мирных жителей в ходе бомбардировок Югославии. Согласно мандату Гаагского трибунала, суд должен был рассматривать все военные преступления на территории бывшей Югославии, совершенные до 2000 года. Убийства американцами мирных жителей безусловно входили в этот мандат. Американцы заставили судей трибунала и прокурора подписать отдельный документ, который гарантировал всем военным НАТО иммунитет от преследования.
	5.536	Бобанич ликвидирован группой российских спецназовцев в пяти километрах южнее города Изюм. Лидер Львовской неонацистской группировки вступил в ряды «Правого сектора» в ноябре 2013 года. С 2014 г. участвовал в карательной операции киевского режима на Донбассе. Лично отдавал приказы на артобстрелы жилых кварталов Донецка и Луганска из тяжелого вооружения. Виновен в сотнях смертей мирных жителей ДНР и ЛНР, в том числе детей.
	5.426	В 2001 году, во время гражданской войны в Македонии, местные офицеры-македонцы говорили, что в конечном счёте, задача шиптаров создать единое государство на обломках Югославии.
	5.193	На Донецком фронте российские войска прорвались на южные окраины Новомихайловки. Также наступают с северо-востока. И там у ВСУ серьёзные проблемы. ВС РФ также продвинулись у Победы, где находится мощный украинский укрепрайон. Атаки у Георгиевки. Здесь без успеха. Гарнизон ВСУ в Авдеевке «горит» на северо-восточных окраинах. ВС РФ вошли в город и пытаются перерезать основную логистическую линию в Ласточкине. Российские войска продвинулись и в районе Степного (не путать с одноимёнными населёнными пунктами на Запорожском и Угледарском направлениях). Также они атаковали в сторону Очеретина. (Рис. 4)
	5.053	Прииде же на оный сход гетман Володька-скоморох в рубище облачен и брадою космат и нача государей немецких аки юродивый стыдити да соромити. Наипаче же со дьяком аглицкаго стрелецкаго Приказа лаялся во грамотех цифирных. Не хощут бо государи стран латинских, кальвинских да люторских Гетманщину в своем воинском союзе видети, яко негоже холопу ровней панам быти.
Input question: Москва заинтересована в преодолении внутриукраинского кризиса мирным путем.
Top-5 lexical search (BM25) hits
	20.745	Но главное - пока что мирным путем.
	18.101	• Москва искренне заинтересована в дальнейшем укреплении всеобъемлющего партнерства и стратегического взаимодействия с Пекином;
	15.047	• МИД КНР о плане Макрона по решению украинского кризиса: Китай поддерживает европейскую сторону в стремлении содействовать мирным переговорам
	14.333	Она отметила, что конфликты должны решаться только мирным путем, добавив, что для европейцев было бы неприемлемо "одностороннее насильственное изменение статус-кво".
	14.208	США следует снять с повестки вопрос о вступлении Украины в НАТО, это станет лучшим путем для урегулирования конфликта на территории бывшей советской республики мирным путем. Такое мнение выразили обозреватели европейской версии издания Politico.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.896	Круг русской истории замкнулся. Перед новым витком.
	0.895	Владимир Зеленский дал комментарий по поводу взрыва на Крымскому мосту:
	0.887	По данным ведомства, задержанные в России исламисты вербовали и переправляли боевиков в Сирию, призывали к терактам.
	0.885	Заместителем министра труда Крыма назначена Гульнара Ваджиповна Ибрагимова, экс сотрудник прокуратуры.
	0.879	Минобороны рассекретило документы о завершении Второй мировой войны.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.532	Мы, конечно, не претендуем на звание лучших организаторов вечеринок. У некоторых, как говорят, выходит веселее. Но чисто донецкий квартирник, с хорошей музыкой, искренними эмоциями и суровыми лицами устроить можем запросто.
	8.465	Все верно. Мы, например, доподлинно знаем, что в некоторых странах от курсантов военно-образовательных заведений в качестве практических заданий требуют вносить поправки в Википедию. Разумеется,  в интересах своей страны.
	8.432	В центре Москвы на Хамовническом валу после ДТП перевернулся автозак с задержанными.
	8.429	Как заявляют участники, мероприятие является противовесом активности прозападных сил в Армении.
	8.407	Как заявил источник РИА Новости в правоохранительных органах, подозреваемый планировал подорвать военкомат.
Input question: При посредничестве России проводятся мирные переговоры на Украине.
Top-5 lexical search (BM25) hits
	23.467	Между тем на госдаче идут переговоры Хаджимба с Бжания при посредничестве Нургалиева.
	19.724	Bild: немецкие депутаты требуют от Шольца начать мирные переговоры с Россией по Украине
	19.700	• Греция и Турция провели тайные переговоры в Брюсселе при посредничестве дипломатического советника Шольца — греческие СМИ
	16.676	Госдепартамент США заявил, что мирные переговоры с Россией по Украине не ведутся, утверждает, что "Москва не демонстрирует к ним интерес"
	16.175	Для скорого завершения конфликта на Украине необходимо незамедлительно начать мирные переговоры, заявил бывший президент США Дональд Трамп во время выступления перед сторонниками в Неваде.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.868	В целом согласен. Одно уточнение. Сегодня на Украине нет пророссийских политиков. Даже условно.
	0.852	 В 15 часов могут начаться российско-украинские  переговоры.
	0.844	Китай примет участие в заседании по Украине, которое пройдет в Саудовской Аравии (и куда не пригласили Россию).
	0.841	В сущности, ничего удивительного. Акции протеста изначально готовились под не принятие итогов выборов и команда проекта
	0.838	Сакуров упрекает Путина: почему звание Героев России носят и погибшие псковские десантники, и сражавшиеся на другой стороне чеченцы, в частности Кадыровы.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.009	Российский постоянный представитель при ООН Василий Небензя заявил, что призывал к прекращению огня и переходу к переговорам.
	8.932	Китай примет участие в заседании по Украине, которое пройдет в Саудовской Аравии (и куда не пригласили Россию).
	8.923	Сегодня внимание россиян, по понятным причинам, приковано к войне на Украине, но из Арцаха, где развёрнут российский миротворческих контингент, поступают тревожные новости.
	8.914	Сакуров упрекает Путина: почему звание Героев России носят и погибшие псковские десантники, и сражавшиеся на другой стороне чеченцы, в частности Кадыровы.
	8.899	Начался очередной раунд переговоров между Россией и Украиной. Сегодня они проходят по видеосвязи.
Input question: Запад и НАТО размещают военную технику вблизи российских границ.
Top-5 lexical search (BM25) hits
	26.529	США создают бронекулак вблизи российских границ
	18.419	Военнослужащие ремонтно-восстановительного батальона группировки войск "Запад" на купянском направлении непрерывно восстанавливают, ремонтируют и модернизируют военную технику и вооружение в ходе спецоперации.
	17.132	Будет ли в дальнейшем Армения, руководствуясь заданным ориентиром на Запад, требовать российских пограничников передать кому-то контроль границ? Будут ли они вообще, после гипотетического ухода российских военных?
	16.393	НАТО проводит учения вблизи российской границы
	15.402	Экипажи с дистанции более 5 км уничтожили позиции националистов и их военную технику.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.868	— Иордания начала переброску тяжёлой техники к западной границе.
	0.859	Военнослужащие ремонтно-восстановительного батальона группировки войск "Запад" на купянском направлении непрерывно восстанавливают, ремонтируют и модернизируют военную технику и вооружение в ходе спецоперации.
	0.857	4. Запрет на строительство газопровода в датских территориальных водах
	0.856	И пример Французской революции, с её гильотинами и адскими колоннами, стучит в сердце наших тоталитарных западников.
	0.854	Записали эксклюзивное интервью Гостомельских десантников, участвовавших в спецоперации.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.626	Завод использовался ВСУ для укрытия вооружений, в частности, гаубиц и минометов. Также на заводе располагался личный состав ВСУ.
	8.451	Некоторое время назад мы все гадали, зачем западные СМИ так яростно раздувают утечку секретных данных. Официальное заявление делал даже Джо Байден.
	8.384	Не уверен, что заход на российскую территорию был запланирован. Тут или просто слегка заплутали и группа эвакуации и диверсанты.
	8.341	В уходящем году Затулин написал законопроект о репатриации и представил его заинтересованным ведомствам. Но не получил ни одного положительного отзыва.
	8.311	Военнослужащие ремонтно-восстановительного батальона группировки войск "Запад" на купянском направлении непрерывно восстанавливают, ремонтируют и модернизируют военную технику и вооружение в ходе спецоперации.
Input question: Киев ежедневно обстреливает Донбасс, в том числе из оружия поставленного НАТО, и готовится к силовому решению конфликта.
Top-5 lexical search (BM25) hits
	23.024	- без поставленного НАТО вооружения
	21.437	Русских людей в Донецке, судя по всему, убивали из поставленного европейцами и американцами оружия.
	19.786	Вообще НАТО к кибервойне готовится крайне серьёзно.
	18.890	Часть оружия они уничтожили, но многое, в том числе и образцы вооружения стран НАТО попало в руки защитников ДНР неповреждённым.
	18.864	Национальная гвардия США готовится к возможным конфликтам с Россией и Китаем в Арктике, в том числе в случае военного столкновения Москвы и Пекина с союзниками Вашингтона по НАТО, сообщает Military Times.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.907	Глава конституционного комитета Совфеда Андрей Клишас заявил, что режим КТО (был введен в Москве, области и Воронежской области) продолжает действовать, поэтому все ограничения нужно соблюдать.
	0.906	• С поставками истребителей Киеву в Европе исчезает грань между косвенной и прямой вовлеченностью в конфликт с РФ;
	0.899	К турецкому городу Эдирне приближаются десятки тысяч нелегалов из стран Африки и Ближнего Востока, именующие себя «Конвоем Света».
	0.898	Сообщается, что на родине Гриффин проживал в Нью-Йорке. Данных об обстоятельствах его смерти нет.
	0.898	СВО показала, что строить корабли надо не быстро, а немедленно, и это уже делается, заявил главком ВМФ РФ Николай Евменов.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.990	Киев активизировал диверсионную деятельность в Запорожской области с начала мая, но силы РФ ей успешно противодействуют, отметил Балицкий.
	6.823	Как отметили авторы статьи Джейсон Горовиц и Кэтрин Портер, европейские страны все еще активно поддерживают киевский режим, но их решимость пошатнули проблемы понижения уровня жизни и инфляции, а также усталость от конфликта, в который вкладывается много ресурсов.
	6.634	Капитан Евгений Козловский посмертно удостоен звания Героя России. Даже после гибели командира бойцы называют свое подразделение «ротой Козловского». О подвиге офицера и настоящем мужестве — в видео.
	6.620	Девиации становятся не нормой даже, а признаком особой отметки, едва ли не свыше. Новые блаженные новых культов это всякие гендерфлюидные небинарные трансформеры.
	6.477	• Евросоюз пока не готов конфисковать замороженные активы РФ, сообщил советник премьер-министра Нидерландов Джеффри ван Леувен. В числе причин он назвал несогласие с этим Германии.
Input question: Цель военной операции Росии на Украине – защита людей от геноцида.
Top-5 lexical search (BM25) hits
	19.092	 На освобождённых от украинского нацистского режима территориях формируются органы внутренних дел. Их цель - защита мирных граждан от противозаконных посягательств.
	18.366	В Сирии проходят акции в поддержку российской военной операции на Украине.
	16.964	Преодоление водной преграды подразделениями ВС РФ, участвующими в проведении специальной военной операции на Украине.
	16.964	• Решение об упреждающей военной операции на Украине было абсолютно необходимым и единственно возможным;
	16.543	🎖Министр обороны России Сергей Шойгу наградил медалями «Золотая Звезда» героев специальной военной операции на Украине.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.852	🎖 Командующий войсками Центрального военного округа генерал-полковник Александр Лапин вручил боевые награды участникам специальной военной операции.
	0.846	Вот так выглядит памятная медаль «Участнику специальной военной операции», учреждённая Федеральной службой войск национальной гвардии России.
	0.845	🎖 Командующий войсками Центрального военного округа генерал-полковник Александр Лапин вручил награды российским военнослужащим, которые проявили мужество и героизм в ходе специальной военной операции на Украине.
	0.841	И не забывать ни в коем случае, что Запад глобально целью видит не поддержку Украины, а ликвидацию России. И потому не отступит.
	0.840	Генпрокуратура России намерена дать оценку заявлениям Зеленского, цитируемым в российских СМИ.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.620	Нет никаких сомнений, что цели и задачи специальной операции на Украине будут выполнены, заявил президент РФ Владимир Путин. Глава государства отметил, что у этой операции цели абсолютно понятные и благородные - помощь народу Донбасса.
	8.601	Не молчит она и на запорожском направлении. Обстрелы позиций ВСУ практически на всей линии соприкосновения. Цель – удары по концентрирующимся украинским войскам. Третье оперативное направление их наступления вполне себе вырисовывается.
	8.534	На здании цитата Ататюрка: "турецкий ребенок, познакомившийся с деяниями отцов, найдет в семье силы для великих свершений". После открытия туда будут водить группы восторженных школьников, слушающих лекции о великой тюркской истории. Особенно впечатляет на фоне нашей истории с Ермаком.
	8.527	ЦРУ открыто заявило о намерении вербовать россиян, недовольных спецоперацией
	8.410	Глава администрации Киреевского района Игорь Цховребов уточнил ТАСС, что в результате инцидента никто не пострадал.
Input question: Украинские пограничники не оказывают никакого сопротивления российским войскам.
Top-5 lexical search (BM25) hits
	22.454	Полностью демотивированный на продолжение сопротивления российским войскам украинский командир просит, чтобы его и их военнослужащих вывезли.
	17.782	Украинские пограничники не выпускают из страны несовершеннолетних мальчиков
	14.829	Срочно⚡️Аваков слил украинские гостайны российским пранкерам
	14.029	Украинские пограничники со стрельбой задержали в Закарпатье военнообязанных, пытавшихся покинуть страну через реку.
	13.774	Кадры захода нациков на КПП "Грайворон" в Белгородской области. Наглядно видно, что никакого "захода без сопротивления" не было - по силам противника велся артиллерийский огонь.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.873	Украина согласится на транзит российского аммиака по своей территории, если получит гарантии от Москвы и ООН о безопасном экспорте зерна через Чёрное море, сообщает Reuters со ссылкой на украинского чиновника.
	0.873	На луганском направлении украинская армия пытается прорваться в районе местного Первомайского и Куриловки в направлении Орлянки и Кисловки. Цель – перерезать рокаду и выйти к участку трассы Р-66. Это основная линия снабжения российской группировки в Донбассе.
	0.869	На луганском направлении уже украинская армия пытается прорваться к трассе Р-66 в районе Краснопоповки и перерезать её участок к северу от Кременной. В свою очередь, российские войска атаковали в направлении Стельмаховки. (Рис. 5)
	0.866	А поскольку приграничное столкновение поставило Россию и Англию на грань войны, то начались санкции, истерики, угрозы, ну и дипломатическая переписка.
	0.865	Наркомания в украинских войсках повсеместна, командиры закрывают на это глаза, рассказал пленный украинский солдат.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.590	Украинская делегация устроила провокацию в отношении российской в ходе мероприятий Парламентской ассамблеи Черноморского экономического сотрудничества в Анкаре – РИА Новости
	8.563	Динамика на угледарском участке. Российские войска в Павловке. Украинские отчаянно пытаются удержаться на левом берегу реки Кашлыгач. Снабжение группировки ВСУ со стороны Угледара проблематично. И часть рокады от Никольского до Пречистовки перерезана. (Рис.4)
	8.512	Украина согласится на транзит российского аммиака по своей территории, если получит гарантии от Москвы и ООН о безопасном экспорте зерна через Чёрное море, сообщает Reuters со ссылкой на украинского чиновника.
	8.486	На Луганском направлении российские войска заняли ряд новых позиций в районе Невского, Макеевки и Ивановки. Атакуют укрепрайон ВСУ в Синьковке. Удары по украинским позициям в Белогоровке, Новолюбовке, Песчаном, Куриловке, Купянске и Петропавловке. (Рис. 5)
	8.480	На луганском направлении украинская армия пытается прорваться в районе местного Первомайского и Куриловки в направлении Орлянки и Кисловки. Цель – перерезать рокаду и выйти к участку трассы Р-66. Это основная линия снабжения российской группировки в Донбассе.
Input question: Военная инфраструктура Украины выведена из строя.
Top-5 lexical search (BM25) hits
	30.868	"Высокоточными средствами поражения выводятся из строя военная инфраструктура, объекты ПВО, военные аэродромы, авиация Вооруженных сил Украины".
	28.776	ФОТО⚡️Васильевская станция выведена из строя
	27.857	- Ладыжинская ТЭС выведена из строя (Винница)
	23.125	Энергетическая инфраструктура Киева может выйти из строя в любой момент.
	20.652	Где-то на Авдеевском направлении бойцами из 1-го АК была выведена из строя, а после – уничтожена, очередная иностранная БМП.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.869	 Укрофашисты стреляют по мирняку на выезде из Мариуполя.
	0.849	Идеология украинства это вот как раз та идеология, которая про убийство детей. Она такая со времен Волынской резни,  со времен Терезина и Талергофа.
	0.848	Только за Украиной еще стоит вся разведывательная, военная и политическая инфраструктура НАТО и англосферы.
	0.842	– Организаторы авиакатастрофы всё сделали не профессионально.
	0.842	На втором видео Газа после сегодняшней атаки Израиля. Как видим, интенсивность конфликта остается на высоком уровне.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.028	Только за Украиной еще стоит вся разведывательная, военная и политическая инфраструктура НАТО и англосферы.
	8.903	Небольшая подсказка: страна классная, но всё время проигрывает информационную войну, чем вгоняет граждан в информационную депрессию.
	8.869	Жена советника Офиса президента Украины Алексея Арестовича Анастасия отказалась в интервью говорить на украинском языке.
	8.868	Теперь оказывается, американцы умеют менять идентификационный сигнал морских судов, а возможно и воздушных.
	8.736	Сегодня днём украинскими РСЗО была обстреляна нефтебаза, которая располагается неподалёку от оживлённой трассы, проходящей через Макеевку.
Input question: Цели спецоперации - денацификация и демилитаризация Украины.
Top-5 lexical search (BM25) hits
	29.920	Это денацификация, демилитаризация, нейтральный статус Украины – В.Путин.
	27.905	Денацификация, демилитаризация и нейтральный статус.
	27.014	Денацификация и демилитаризация бандеровцев выглядит так.
	17.125	 Демилитаризация Украины. В  Мелитополе первые жители начали добровольно сдавать оружие.
	15.046	• Цели и задачи спецоперации не меняются после атаки беспилотников на Кремль;

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.862	Уроки русской истории от  ценного иностранного специалиста.
	0.858	• РФ высоко ценит позицию Сербии, которая не присоединилась к санкциям в условиях беспрецедентного давления;
	0.857	На днях целый спектр эмоций от непонимания до возмущения, вызвала ситуация с выдачей российских паспортов беженцам из Палестины.
	0.853	❗️ Отставной офицер командования Сил Специальных Операций ВСУ Цигипа Сергей Витальевич рассказал о работе подразделений информационно-психологической операции на Украине.
	0.852	Все цитаты нанесены в стиле традиционной русской каллиграфии.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.279	Это денацификация, демилитаризация, нейтральный статус Украины – В.Путин.
	8.269	На днях целый спектр эмоций от непонимания до возмущения, вызвала ситуация с выдачей российских паспортов беженцам из Палестины.
	8.223	❗️ Отставной офицер командования Сил Специальных Операций ВСУ Цигипа Сергей Витальевич рассказал о работе подразделений информационно-психологической операции на Украине.
	8.223	• Спецоперация более чем оправдана, нужна и должна завершиться достижением поставленных целей.
	8.202	Первоначальные планы израильской операции вызвали опасения у американских официальных лиц, которые считают что у операции отсутствуют достижимые военные цели, говорится в статье.
Input question: Основная задача спецоперации — «освобождение Донбасса».
Top-5 lexical search (BM25) hits
	19.324	Основная задача ЗРК — противовоздушная оборона позиций российских войск, в частности от вражеских беспилотников.
	15.986	• российские войска в зоне СВО держат активную оборону, основная задача - выбить технику противника
	15.590	Наступление Русских войск на Донбасском фронте продолжается. Основная стратегическая задача – прорвать линию Северск-Соледар-Бахмут-Часов Яр.
	15.404	Минобороны России объявило о создании штаба по гуманитарному реагированию на ситуацию на Украине, его основная задача — оказание гумпомощи населению ДНР, ЛНР и Украины
	13.442	Видео⚡️«Купол Донбасса» наступает на Невельское

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.813	Вспомнил героев из нашего фильма "Освобождение". Бойцов армейского корпуса ДНР.
	0.804	Спецрепортаж WG⚡️Защитники Запорожья: направление Орехов – Работино
	0.800	Все подробности в нашем специальном включении из освобождённых дворов Левобережного района.
	0.796	 Кадры задержания сотрудника ОБСЕ,  передававшего секретные данные иностранным спецслужбам.
	0.785	Зверьё нацистское. На освобождённой от нацистов с 57 бригады территории в районе села Кряковка Славяносербский район, был обнаружен

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.507	При задержании ДРГ оказала сопротивление, завязался бой. В результате боестолкновения ранены двое донецких спецназовцев. Данные о потерях среди диверсантов не сообщаются.
	8.410	Служба безопасности Швеции заявила, что при осмотре места ЧП на "Северных потоках" задокументированы обширные повреждения.
	8.344	Боевики «Дивизии Султана Мурада», отправленные турецками спецслужбами в Азербайджан, «зарекомендовали» себя в Сирии участием в военных преступлениях и обстрелами христианских кварталов Алеппо
	8.333	Замглавы комитета Совета Федерации по международным делам Джабаров – об ответе США и НАТО на российские требования безопасности:
	8.328	Как известно, в дни наступления ВСУ на Ореховском направлении Запорожской области, неоценимую поддержку обороняющимся оказывала авиация.
Input question: Задача спецоперации - обеспечить гарантии безопасности России.
Top-5 lexical search (BM25) hits
	17.785	Также Макрон заявил, что будущая архитектура безопасности в Европе должна включать гарантии для России.
	17.522	Гарантии безопасности для Украины — гарантиия безопасности всей Европы — Ермак
	15.777	Тут наш посол в США заявил, что России нужны железобетонные гарантии безопасности, но американцы пока не готовы к диалогу.
	15.096	Вместе с этим французский лидер продолжает возлагать надежды на дипломатию: России и другим странам постсоветского пространства необходимо предоставить новые гарантии безопасности.
	14.840	Главная задача наших военных - обеспечить режим прекращения огня. И неважно какие доводы для этого используются.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.851	Спасибо, Григорий, за зоркий операторский глаз!
	0.841	Коалиция стран Запада, направленная против России, начала слабеть из-за экономических трудностей и опасений, что кризис на Украине затянется, сообщает The New York Times.
	0.834	Рамзан Кадыров показал кадры отправки из Чечни новой группы добровольцев в зону спецоперации.
	0.826	На Украине заработало "Радио России". Запись вещания на частоте 999 КГц в городе Энергодар Запорожской области.
	0.824	• В конце марта польский генштаб организовал учения по отработке действий в случае нападения с запада.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.438	Коалиция стран Запада, направленная против России, начала слабеть из-за экономических трудностей и опасений, что кризис на Украине затянется, сообщает The New York Times.
	8.206	Президент Турции также заявил, что операции ВС Турции на севере Сирии и Ирака являются "только началом", Анкара решительно настроена обеспечить безопасность границ.
	8.186	Насколько может обеспечить Запад безопасность Армении от возможности агрессивных действий Азербайджана и Турции вопрос, конечно, интересный. И самое главное, согласится ли на это Иран – это, вообще, отдельный разговор.
	8.151	Рамзан Кадыров показал кадры отправки из Чечни новой группы добровольцев в зону спецоперации.
	8.109	Захваченный палестинцами израильский генерал, это не просто абы кто, а командующий одним из подразделений спецназа.
Input question: Успешное завершение «первого этапа спецоперации».
Top-5 lexical search (BM25) hits
	12.223	Успешное выполнение боевых задач российскими летчиками на направлении V
	12.086	Ну что, яркое такое завершение условно выходного дня в России.
	11.800	Министр обороны Израиля назвал три этапа войны с ХАМАС
	11.240	В бою легче не будет, но полученные навыки обеспечат успешное выполнение задач.
	10.855	Советник главы ЛНР Родион Мирошник о том, каким он видит завершение спецоперации на Украине.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.831	В Краматорске с завода "Энергомашспецсталь" после вчерашнего
	0.814	Мощные удары нанесены по району завода "Энергомашспецсталь" в Краматорске, где засели военные ВСУ
	0.804	Зачем это делать на десятый месяц СВО совершенно непонятно.
	0.796	Вечная слава героям, павшим в боях при решении задач специальной военной операции!
	0.795	Но главная эвакуация, конечно, прошла мега-успешно.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.617	После начала спецоперации, преступлений стало меньше, но сейчас засевшие на Украине преступники готовят новую волну дистанционных мошенничеств, для чего вновь вербуют беспринципных россиян, готовых совершать преступления.
	8.538	Лучший завтрак - это завтрак на свежезавоёванных украинских позициях. Даже прилёты 120-х мин не в состоянии повлиять на аппетит «спартанцев». Авдеевский котёл. Наши дни.
	8.409	Мощные удары нанесены по району завода "Энергомашспецсталь" в Краматорске, где засели военные ВСУ
	8.306	Байден заявил, что готов вести переговоры с Путиным о завершении российской спецоперации на Украине, если тот примет такое решение.
	8.278	Это Рязань прошлой ночью. Жители радостно приветствуют своих защитников, бойцов, выполнявших задачи в зоне проведения специальной военной операции.
Input question: Произошло «значительном снижении военного потенциала Украины».
Top-5 lexical search (BM25) hits
	19.207	• Западу нужен план долгосрочного укрепления военного потенциала Украины;
	17.662	• Киевский режим использовал сделку для наращивания военного потенциала и военной инфраструктуры.
	15.574	Нет никаких критических разрешений, которые привели бы к коллапсу экономики или хотя бы ослаблению военного потенциала Украины.
	14.871	• Новый пакет военной помощи США Украине включает $225 млн на долгосрочное укрепление ее военного потенциала и модернизацию ВСУ;
	14.229	Советник главы офиса Зеленского Подоляк заявил, что единственный путь к завершению конфликта - это максимальное усиление военного потенциала Украины и "возвращение территорий"

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.859	• Россия признательна Китаю за взвешенную линию по ситуации на Украине, за понимание предыстории и подлинных причин;
	0.846	• Россия признательна Саудовской Аравии за взвешенную позицию по ситуации на Украине.
	0.840	 В Подмосковье задержан сторонник нацистской идеологии за поджог полицейской машины с символикой в поддержку военной операции на Украине.
	0.840	На созванном по инициативе России заседании Совбеза ООН по Украине сегодня он может выступить как докладчик.
	0.837	Задержан в аэропорту Внуково при попытке сбежать из России. Италию ушлепок увидит очень нескоро.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.698	Уволен за свою открытую политическую позицию, за поддержку нашей армии и спецоперации.
	8.628	Сегодня ночью на Запорожской АЭС, в результате устроенной ВСУ провокации, произошло возгорание учебного и административного корпуса.
	8.582	• Россия признательна Китаю за взвешенную линию по ситуации на Украине, за понимание предыстории и подлинных причин;
	8.560	На созванном по инициативе России заседании Совбеза ООН по Украине сегодня он может выступить как докладчик.
	8.544	Зеленский заявил, что украинские специалисты примут участие в расследовании инцидента с ракетой в Польше.
Input question: Россия ведёт оборонительную войну против НАТО.
Top-5 lexical search (BM25) hits
	25.276	🤡 Россия ведёт войну у порога НАТО — президент Литвы Гитанас Науседа
	22.622	Сегодня народ ведёт войну против злейшего врага.
	19.009	Он выиграл тяжёлую трёхлетнюю войну против средневекового НАТО.
	17.166	Устройство НАТО и безопасность Европы в целом – Россия выиграла войну нокаутом.
	15.488	"Украина не наносит ударов по территории РФ. Украина ведет оборонительную войну с целью деоккупации всех своих территорий", написал Подоляк в Twitter.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.827	Не надо отвечать на оскорбления оскорблениями.
	0.827	Но Следственный комитет отчего-то состава преступления не видит.
	0.827	Рабочая партия Курдистана отрицает обвинения в свой адрес.
	0.820	Ситуация в Авдеевке и Марьинке контролируемая, опасаться серьезного продвижения ВСУ не стоит.
	0.820	• Россия стала сильнее, потому что все россияне вместе, "за нами правда, а в правде — победа".

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.107	Не Россия виновата, что Арменией руководит предатель. Россия виновата в том, что позволяет терпеть оскорбления в свой адрес.
	9.019	Рост населения дает рост ВВП, но НЕ гарантирует при этом рост доходов населения, почему написал выше в посте про производительность.
	8.944	Ну собственно, российская армия (сначала советская) всегда готовилась именно что к противостоянию с НАТО.
	8.815	В-третьих, ни в Крыму, ни в восточной половине Донбасса, которую захватила Россия, не было ни восстания, ни вооруженного сопротивления.
	8.748	Военкоматы страны получили право вести круглосуточную работу в условиях мобилизации и военного положения – об этом заявил представитель украинского правительства в Раде.
Input question: НАТО накачивает Украину оружием.
Top-5 lexical search (BM25) hits
	16.761	В смерти Никиты виноваты не только украиские мясники-артиллеристы. Не только ублюдки из НАТО, которые накачивают Украину оружием.
	15.303	Видео с самым передовым оружием НАТО,
	13.164	В Греции Зеленского назвали «кровавым диктатором» и потребовали прекратить снабжать Украину оружием ❗️❗
	12.678	Сегодня Украину вновь не примут в НАТО — Bloomberg
	12.318	Зеленский призвал Байдена пригласить Украину в НАТО на грядущем саммите

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.878	Также в Вашингтоне не считают, что Москва намерена применить ядерное оружие на Украине.
	0.839	На этом видео отчётливо видно, кто на самом деле переживает за Украину.
	0.838	Срочно⚡️Леопарды уже замечены на Украине в Николаеве
	0.831	В смерти Никиты виноваты не только украиские мясники-артиллеристы. Не только ублюдки из НАТО, которые накачивают Украину оружием.
	0.830	Уважаемый Николай!

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.507	Уже давно не секрет, что американцы напичкали Украину своим вооружением. Но не только им. Украинскую армию оснастили и военными автомобилями. Но в Мариуполе ничто не спасло их от русского оружия. С помощью квадрокоптеров хаммеры были обнаружены, и по ним был нанесён удар.
	9.436	Но, как нас успокаивают чиновники рангом пониже, с этим придётся смириться, мигранты нам очень сильно нужны, якобы.
	9.382	У нас нет возможности одновременно обычными вооружениями противостоять НАТО на Севере, на Балтике, Чёрном море, Тихом океане. А стратегическими силами можем.
	9.296	Но ещё раз скажу: враг очень умело работает. Понимает, какой психотип нужен для вербовки. Думаю, что это даже не ГУР, думаю, что западные разведки составляли методику.
	9.273	Я на самом деле, даже не понимаю, что сейчас чувствуют украинские военные. Не всякие упоротые нацисты, а обычные мужики. Русские, зачастую, люди.
Input question: Противостояние с Североатлантическим альянсом.
Top-5 lexical search (BM25) hits
	13.075	• Россия воспринимается альянсом как враг, как противник
	11.939	С утра пораньше генсек НАТО не исключил перерастания событий на Украине в войну между альянсом и РФ.
	11.446	• Киев нацелен на втягивание Кишинева в жестокое противостояние с РФ;
	11.214	• США с союзниками провоцируют другие страны на военное противостояние с РФ и КНР;
	10.804	Удар по НАТО: Минобороны России заявило об уничтожении арсенала, переданного альянсом для ВСУ в Одессе.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.862	Светлой памяти, Арсен.
	0.829	Активность ВВС США в Европе прямо сейчас.
	0.824	Уточняется, что исключение составляют те случаи, когда предоставление персональных данных является обязательным в соответствии с законом.
	0.824	Со СМИ все ясно, идем читать первоисточник. -
	0.818	Пасечник вряд ли бы стал пугать Киев наступлением, предварительно не согласовав свою позицию с Москвой.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.287	Перечитываю переписку с Егором. Самое интересное, наверное, стоит выкладывать. Пару месяцев назад обсуждали с ним перспективы православной церкви и возможно ли церковному руководству привлечь активную молодежь. Просвирнин пишет:
	9.270	Понятно, Европа сейчас рассчитывает, что Россия просто дрогнет, испугается. На фоне активности ВСУ и удара в Харьковской области.
	9.175	Спустя 15 месяцев пандемии коронавируса становится очевидно, что Россия пережила её с наименьшими потерями.
	9.159	Причём, откуда вдохновение черпается, понятно. Весь этот университетский необольшевизм, BLM, раскол по линии левацкая молодёжь против среднего класса.
	9.143	Правнук Николая Первого. Смертельно ранен при наступлении на Владиславов в бою с немецкими кавалеристами.
Input question: Украина обстреливает Донбасс снарядами калибра 155 миллиметров.
Top-5 lexical search (BM25) hits
	24.507	• Военные круги в Брюсселе сомневаются в плане, по которому ЕС должен передать Украине миллион снарядов калибра 155 миллиметров — Welt am Sonntag
	22.596	Германия за неделю поставила Украине 10 танков Leopard 1 и свыше 3 тысяч снарядов калибра 155 миллиметров, следует из обновленного списка на сайте министерства обороны ФРГ.
	22.325	По данным "Известий", предварительно, обстрел велся снарядами натовского калибра 155 мм. Глава администрации в этот момент находился в кабинете, он получил легкую контузию.
	22.072	Предположительно, работала вражеская ствольная артиллерия 155 калибра.
	20.230	• Германия за неделю поставила Украине две пусковые установки системы ПВО IRIS-T модификации SLS, а также свыше 4,3 тысячи дымовых боеприпасов калибра 155 миллиметров, следует из обновленного списка на сайте правительства ФРГ.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.845	 Идёт обстрел Киевского и Куйбышевского районов г Донецка. Удары наносятся артиллерией калибром 155мм ( нато)
	0.834	СРОЧНО⚡ВСУ рано утром обстреляли дорогу, ведущую из Донецка в Яковлевку, из 120мм миномётов.
	0.829	Эксклюзивное интервью руководителя СДД Александра Бородая по итогам совета командиров добровольцев Донбасса.
	0.827	Легенда об императоре Константине в трудах российских византиноведов. 700 рублей.
	0.827	К такому выводу украинская сторона пришла после обнаружения на обломках дронов маркировки, выполненной кириллицей.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.052	Украина и Польша договорились о совместном производстве танковых снарядов калибра 125 мм, заявили в Киеве
	7.915	Украина с февраля прошлого года получила от партнеров военной помощи в денежном эквиваленте равном 100 миллиардам долларов, заявил министр обороны страны Алексей Резников.
	7.909	Два украинца пытались привезти в Россию почти 200 килограммов наркотиков. Они спрятали упаковки в днище автомобиля и поехали транзитом через Латвию. Псковские таможенники проверили транспортное средство, в котором находилось 247 брикетов с наркотиком. Масса — 186 килограммов.
	7.907	К такому выводу украинская сторона пришла после обнаружения на обломках дронов маркировки, выполненной кириллицей.
	7.862	• Когда Украина получит достаточную противовоздушную оборону, главная российская угроза будет остановлена.
Input question: Поставляя Украине оружие, страны НАТО «играют с огнем», они ничего не меняют, а только затягивают конфликт.
Top-5 lexical search (BM25) hits
	22.483	Конфликт на Украине не изменил НАТО,
	18.778	Страны НАТО должны расширить свое военное производство, чтобы одновременно поставлять оружие Украине и восполнять собственные запасы — Столтенберг
	18.271	Он отметил, что партнеры не только передают собственное вооружение, но и работают с третьими странами, покупая у них оборудование и поставляя его Украине скрытно.
	17.926	Израиль не может поставлять Украине оружие, как это делают страны НАТО, так как сам регулярно ведет операции с участием войск, заявил посол Израиля в Киеве Михаэль Бродский.
	17.192	Поставляя американское оружие Киеву, мы становимся виновниками гибели тысяч человек — американский конгрессмен Пол Госар

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.833	Поспорю немного. Сейчас потрясающая молодёжь, которая не меняет дедовскую медаль "За отвагу" на джинсы, не палит кнопки в лифте, не проламывает тебе череп, если ты не с их района.
	0.826	По его словам, в случае вооруженного конфликта с РФ "никто не будет воевать" на стороне Грузии. "Мы будем одни. Мы смотрим на Украину и видим, что никто не борется рядом с украинским народом или вместо него. Максимум, что может сделать Запад, так это поставить оружие. Они оружие-то поставляют, однако на украинской земле льется кровь лишь украинцев", - сказал Папуашвили.
	0.825	 «Наверное, именно сегодня это важно отметить. Выдохся миф, который выстраивала вокруг второй мировой войны советская, а потом российская пропаганда. Причём он протух уже не только для внешнего, но и для внутреннего использования».
	0.816	 У нас с инфополем в Интернете дела обстоят, на мой взгляд, как борьбой за историческую память.  Вот памятник Коневу в Праге снесли. Мы осуждаем. На самом высоком уровне. И правильно делаем. Иначе невозможно. И в Россию надо памятник забрать.
	0.814	Так вот нет ничего удивительного и в том, что с талибами сначала дружили, потом против них воевали, теперь сдают им страну. Очень по англо-саксонски. История Первой Англо-афганской. Один в один.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.290	Мне кажется пора уже родителям вернуться. И я не призываю к новому военному конфликту. Я предлагаю начать наказывать или поощрять руководство Латвии за правильную или не правильную политику.
	8.100	Почему по нефтехранилищам ударили в качестве возмездия, а не в ходе плановой работы по уничтожению военной инфраструктуры противника. Ему, например, не нужны поводы, чтобы этим заниматься системно. И он занимается, не стесняясь.
	7.972	По его словам, в случае вооруженного конфликта с РФ "никто не будет воевать" на стороне Грузии. "Мы будем одни. Мы смотрим на Украину и видим, что никто не борется рядом с украинским народом или вместо него. Максимум, что может сделать Запад, так это поставить оружие. Они оружие-то поставляют, однако на украинской земле льется кровь лишь украинцев", - сказал Папуашвили.
	7.947	Я написала заявление в СК , что Сбербанк не реагирует ,что на карту Сбербанка РФ идёт сбор денег для поддержания террористической деятельности против войск РФ. Ничего:((
	7.935	Поспорю немного. Сейчас потрясающая молодёжь, которая не меняет дедовскую медаль "За отвагу" на джинсы, не палит кнопки в лифте, не проламывает тебе череп, если ты не с их района.
Input question: Транспорт с поставками оружия на Украину будет законной целью РФ.
Top-5 lexical search (BM25) hits
	19.342	• Любые поставленные Украине вооружения станут законной целью для российских военных;
	17.682	• Поставками оружия США подстрекают Киев "к новым ужасным деяниям";
	15.165	• Слова Нуланд о том, что Крым является законной целью для Киева и США, подчёркивают глубину разногласий Соединённых Штатов и России.
	15.077	Швейцария не будет разрешать реэкспорт своего оружия на Украину, пока парламент не примет соответствующего решения — постпред в ООН
	14.893	«Исламское сопротивление Ирака» взяло на себя на ответственность за нападение, заявив, что все американские позиции в регионе станут законной целью, пока «преступления сионисткого режима не прекратятся».

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.906	Как вариант , в рамках реформы УФСИН надо повестить по всем зонам камеры , которые будут вести трансляцию прямо в Управлении собственной безопасности.
	0.901	Эксперты ЕК пишут, что, с юридической точки зрения, активы ЦБ РФ нельзя просто так передать Украине.
	0.898	🇷🇺🇺🇦🖇 А вот и тезисы украинской стороны для отработки в информационном пространстве: у ВС РФ заканчиваются оружие и боеприпасы, они будут просить помощь у мирного населения.
	0.896	Оператором была зафиксирована эвакуация тела уничтоженного боевика.
	0.896	Точно в цель. Кадры уничтожения украинского танка бойцами ВС РФ. Экипаж спасается бегством.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.722	Как вариант , в рамках реформы УФСИН надо повестить по всем зонам камеры , которые будут вести трансляцию прямо в Управлении собственной безопасности.
	8.646	Также сегодня ночью впервые зафиксирована атака беспилотниками Липецкой области. Силами ПВО был сбит один украинский БПЛА самолетного типа.
	8.631	Также он сообщил, что Россия запускает свой цифровой рубль, который будет неподвластен никаким ограничениям.
	8.627	 Если удары по территории РФ продолжатся, ВС РФ будут нанесены удары по центрам принятия решений, в том числе в Киеве — Минобороны РФ.
	8.600	• РФ будет вынуждена реагировать на поставки Киеву обедненного урана, так как Запад начинает применять оружие с ядерным компонентом.
Input question: Санкции - это «экономическая война, подобной которой еще не было».
Top-5 lexical search (BM25) hits
	13.435	Всеобщих эпидемического характера восстаний между народностями велико- и белорусского типа не было».
	12.867	Рябит от заголовков о новой войне. Во-первых это еще не война с Йеменом. Во-вторых и не с Йеменом вовсе.
	12.860	Еще раз - они герои. Их работа - война, но их гибель это трагедия. Тем
	12.860	Еще раз - они герои. Их работа - война, но их гибель это трагедия. Тем
	12.501	Война, это не шутки.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.835	Это не Послание. Это демонстрация военной мощи России. И такой мощью должен управлять не торговец, не женщина, не эмоционально неустойчивый человек. А понятно кто.
	0.827	Это были эксклюзивные кадры тренировок по ножевому бою российского спецназа
	0.816	Буровая эксклюзивно публикует обзор «Энергодайджеста» Энергетического центра компании EY.
	0.814	А эти еще долго будут лаять на русских из эмиграции.
	0.811	Над Винницкой областью, предположительно, замечен «иранский экспресс». Ранее там видели ракеты.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.436	Это не Послание. Это демонстрация военной мощи России. И такой мощью должен управлять не торговец, не женщина, не эмоционально неустойчивый человек. А понятно кто.
	8.349	По сути, именно на практиках эксплуатации рабочей силы в Бельгийском Конго и была построена система концлагерей в нацистской Германии.
	8.250	❗️ Сенатор Клишас сообщил, что экстремистами не будут считаться владельцы аккаунтов в соцсетях Meta и те, кто продолжат общение на платформах компании, несмотря на блокировки.
	8.225	Полная версия национального позора Франции будет этим вечером на Россия 1. А также опубликуем на канале.
	8.159	Ну а для более рациональной публики транслировать процветание в кущах могучей Империи, как это было во времена РИ.
Input question: Долгосрочная стратегия Запада - «политика сдерживания и ослабления России».
Top-5 lexical search (BM25) hits
	20.362	Важно заметить, что в планах Запада использовать, в первую очередь, страны постсоветского пространства. Ибо их совсем не жалко. Потому что они лишь инструмент для сдерживания и ослабления России.
	16.491	В Минске заявили о реализации мер стратегического сдерживания в ответ на действия Запада
	13.949	«Политика США слишком жёсткая, из-за чего [другие] страны хотят отказаться от доллара.
	12.058	Срочно⚡️Россия начала учения сил ядерного сдерживания
	11.895	Лиддел Гарт "Стратегия непрямых действий".

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.878	Зачистка в секторе ВПК: ряд руководителей Госкорпораций, занимающихся производством оружия, оказались под угрозой увольнения
	0.877	Западные издания обсуждают значение освобождения Соледара российскими войсками:
	0.864	• Политика Запада, направленная на тотальное сдерживание России, крайне опасна, она несет риски скатывания к прямому вооруженному столкновению ядерных держав.
	0.854	⚡️ Россия и Украина обменялись задержанными пограничниками.
	0.847	По предварительной информации, пострадали два человека. Дебошира задержали уже около другого заведения.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.426	• Словакия и Украина договорились о создании системы лицензирования для поставок зерна, запрет на импорт будет действовать до ее запуска.
	8.408	По сюжету, Запад сам предложит России сделку. В итоге российская сторона получит земли от Харькова до Одессы и с нее снимут все санкции. Взамен Россия возьмет на себя обязательство поставлять нефть и газ в Европу на протяжении 15 лет.
	8.376	Злоумышленника задержали в момент совершения сделки. К продаже он предлагал два пистолета, относящиеся к категории огнестрельного оружия, а также боеприпасы к ним. В ходе допроса подозреваемый рассказал, что является коллекционером.
	8.369	Старая история: Запад прощупывает красную линию, до которой, в своей решительности, готова продвинуться Россия.
	8.222	Лидер партии «Справедливая Россия — За правду» Сергей Миронов предложил лишить французского актёра Жерара Депардье гражданства. Он считает, что знаменитости,
Input question: От санкций страдает не Россия, а в первую очередь США и Европа.
Top-5 lexical search (BM25) hits
	26.010	Или, рассказывая в залихватском тоне о проблемах в экономике ЕС, они формируют иллюзию, что Европа от санкций страдает больше, чем Россия, и вообще вот-вот дрогнет и откажется от Украины.
	24.055	Европа, конечно, страдает от санкций и сильно. Но она не развалится точно, люди будут терпеть - революции никакой не будет, а от поддержки санкций ЕС не откажется, потому что вообще не является самостоятельной структурой. Как и отдельные европейские страны не имеют субъектности.
	20.513	Встречал хозяйку степанакертского кафе, пожилую мудрую женщину, которая в первую войну прятала маленького сына в подвале, а сегодня он – на передовой. Вздохнув, она сказала, что в первую очередь от войны страдает мирное население – «с обеих сторон».
	19.042	Гуманитарная помощь США в Сирии отправляется в первую очередь к боевикам
	19.004	Европа не сможет заменить Украине помощь от США

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.846	Есть сомнения, что это война НАТО против России - послушайте.
	0.845	Как сейсмические волны от турецкого землетрясения прошли по странам Европы.
	0.839	— США считают, что операция Израиля в ее нынешнем виде продлится до января.
	0.838	По словам очевидцев, есть пострадавшие - двое раненных.
	0.827	Спустя 15 месяцев пандемии коронавируса становится очевидно, что Россия пережила её с наименьшими потерями.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.747	Санкции США в отношении Сирии препятствуют оказанию помощи пострадавшей от землетрясения стране. Кто помогает Сирии, а кто боится
	8.573	Ереван опасается санкций со стороны Запада за сотрудничество с Россией, дал понять премьер-министр Армении Никол Пашинян в интервью чешскому журналу Respekt.
	8.563	Прибывший спецназ Британии и ряда других стран предназначен в первую очередь для обеспечения эвакуации персонала и архивов посольства.
	8.536	Стоить отметить, что на прошлой неделе Молдова присоединилась к пакету санкций ЕС против России. Так, постепенно, от плавного прекращения сотрудничества, Молдова скатывается в неприкрытую агрессию.
	8.503	Росгвардеец, вернувшийся из зоны проведения специальной военной операции на Украине, сделал предложение руки и сердца своей девушке.
Input question: Об оккупации Украины речи не идет.
Top-5 lexical search (BM25) hits
	18.707	• Речи о повышении налогов сейчас не идет;
	17.563	Речи об организованном противостоянии между христианами и мусульманами пока что не идет. Однако на месте уже были замечены силы «Хезболлы».
	14.386	• Делом о крушении самолета Пригожина занимается СК, это российское расследование, о международном речи не идет, заявил Песков.
	13.965	Речи о ликвидации важной персоны тоже не идет. Важные персоны на передовой, конечно, бывают, но на постах по ночам не дежурят.
	13.965	Ни о каких высокотехнологичных отраслях, где срок окупаемости 10-15 лет или где он не гарантирован или слабо подсчитываем, речи не идет.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.824	Обычней некуда.
	0.821	Украинские неонацисты снова обстреляли РСЗО Киевский район Донецка. Один из ударов пришёлся на травматологическое отделение.
	0.817	Их удалось обменять накануне ночью, сообщили в Народной милиции республики.
	0.812	 Убежден, что столичная полиция, традиционно отработает как надо, и задержит остальных членов группы, принимавших участие в нападении.
	0.807	Успехи россиян при новом наступлении у Лимана и Авдеевки "очень скудны" — координатор стратегической коммуникации Совета по национальной безопасности США Джон Кирби

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	9.240	Украинские неонацисты снова обстреляли РСЗО Киевский район Донецка. Один из ударов пришёлся на травматологическое отделение.
	9.077	Обычное одесское утро. Десять сотрудников военкомата настойчиво учат любить родину несознательного украинца.
	9.073	Отряд добровольцев из Калмыкии прибыл на территорию Украины, чтобы принять участие в специальной военной операции по денацификации.
	9.073	Отряд добровольцев из Калмыкии прибыл на территорию Украины, чтобы принять участие в специальной военной операции по денацификации.
	9.068	Рамзан Кадыров сообщил об отправке очередной группы добровольцев для участия в спецоперации на Украине
Input question: Вхождение «новых регионов» в состав России.
Top-5 lexical search (BM25) hits
	17.704	И тут самое время напомнить, что депутат Олег "палка для селфи" Матвейчев если что, не голосовал за вхождение новых республик в состав России.
	17.483	Ребят, так это тот самый депутат Матвейчев, который не голосовал за вхождение новых республик в состав России. Отсутствовал в исторический день в стенах парламента.
	15.230	Ну а пока вся Россия который день празднует вхождение в свой состав новых территорий, Армения продолжает свой уверенный демарш в сторону Запада.
	14.615	G7 никогда не признает вхождение ДНР, ЛНР, Запорожской и Херсонской областей в состав РФ, как и в случае с Крымом и Севастополем — заявление
	14.214	вхождение в день - переход через альпы

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.826	Но не в современной России.
	0.825	Семён Пегов. Новые стихи.
	0.800	«Минский разворот» похоже начинает играть новыми красками.
	0.796	Денис «Синий» Синенков рассказал в интервью своему старому другу Семёну всё, как на духу. Смотрите уже завтра в эфире нашего военно-исторического шоу ЧВК «Пегов».
	0.795	А также преждевременные разговоры и заявления с обеих сторон о виновности или невиновности кого-то из участников трагедии.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.924	Про репортаж «Новой» из Херсона надо знать только то, что в тексте на без малого 50 000 знаков (!) нет фамилии Слободчиков. В городе, в котором нет боевых действий, среди бела дня украинскими диверсантами был убит местный предприниматель, пытавшийся наладить взаимодействие с российской армией. Но корреспонденту НГ до этого нет никакого дела.
	8.800	Не тратьте деньги своих неизвестных спонсоров на расследование нашей работы в Белоруссии. Расследовать там нечего.
	8.789	Российских политиков измерили «в телеграмах». Эксперты политагентства Г.У.Р.У., обучившие свою нейросеть «НеДуров» ловить все упоминания по ключевым персонам в Telegram, представили cвежий
	8.783	Введение военного положения в новых регионах России - это явно новый шаг к развитию СВО.
	8.776	Прочитал странное в заявлениях некоторых российских политиков, а именно: никаких переговоров после уничтожения Ил-76, с действующим украинским режимом быть не может.
Input question: Цель спецоперации - защита мирного населения от Украины и НАТО.
Top-5 lexical search (BM25) hits
	19.000	• Накачивание Украины западным оружием приведет лишь к увеличению жертв среди мирного населения.
	18.205	От редакции хотим добавить - и это еще без ужасов войны для мирного населения!
	18.143	 На освобождённых от украинского нацистского режима территориях формируются органы внутренних дел. Их цель - защита мирных граждан от противозаконных посягательств.
	17.277	Предварительно, среди мирного населения никто не пострадал. Официальной информации от властей региона пока нет.
	17.050	• Недопущение вступления Украины в НАТО по-прежнему остается одной из целей спецоперации;

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.859	ЦРУ открыто заявило о намерении вербовать россиян, недовольных спецоперацией
	0.848	Уроки русской истории от  ценного иностранного специалиста.
	0.843	• Недопущение вступления Украины в НАТО по-прежнему остается одной из целей спецоперации;
	0.837	Наконец-то Навальный вполне определился в своем отношении к событиям, происходящим на Донбассе. По его словам, уничтожение киевскими властями мирного населения и инфраструктуры - "Это Путин мстит Украине за
	0.837	• Западное вооружение принципиально не изменит исход конфликта на Украине и траекторию достижения Россией целей спецоперации.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.339	Не молчит она и на запорожском направлении. Обстрелы позиций ВСУ практически на всей линии соприкосновения. Цель – удары по концентрирующимся украинским войскам. Третье оперативное направление их наступления вполне себе вырисовывается.
	8.260	С Олега Тинькова сняли санкции. Санкции в целом дело тупое, произрастающее из глубоко колониального сознания Запада.
	8.187	В центре Тбилиси снова начались столкновения между спецназом и участниками акции протеста.
	8.186	ЦРУ открыто заявило о намерении вербовать россиян, недовольных спецоперацией
	8.180	Спецназ полиции в Тбилиси вытеснил митингующих с центрального проспекта Руставели – РИА Новости
Input question: Во всем мире радикалы устраивают массовые беспорядки и свергают (или пытаются свергнуть) легитимные правительства: в Сирии, Ливии, Украине, Беларуси, Казахстане, Венесуэле. За этим стоит Запад.
Top-5 lexical search (BM25) hits
	21.604	Бойцы ЧВК: В Сирии, Украине, Белоруссии. История и реальность. Стоит оно того или не стоит?
	19.827	Так что охранять во имя мира во всем мире, прогресса, счастья всех детей на свете, или во имя интересов страны - дело легальное, нужное и почетное. Чтобы никто не думал, что можно делать всякие безобразия. В Сирии например. Или Африке.
	17.438	Предстоящие переговоры лидеров России и США вызвали огромный ажиотаж во всем мире.
	17.415	"Если Украина победит, то это укрепит ЛГБТ-сообщество во всем мире", —
	17.159	Я видел своими глазами две революции на Украине, революцию в Киргизии, революции в Ливии, Египте, Ливане и Венесуэле. И вот что хочу сказать. Если вы хотите революции в России, то вы идиот.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.870	Министру задали вопрос, не считает ли она, что атаки беспилотников на российские города могут привести к эскалации. "Россия напала на Украину, поэтому у Украины есть закрепленное в Уставе ООН право на защиту своей территории и своих людей", - считает Бербок. Она также утверждала, что "Россия подвергает бомбардировкам гражданские цели". "Украина защищается в рамках международного права", - полагает Бербок.
	0.865	Логика закона вызывает вопросы — ароматизаторы входят в состав многих табачных продуктов, но запретить предлагается только жидкости для вейпов. Если его примут, мы наверняка увидим, как бывшие легальные продавцы уходят на серый рынок и продают товары сомнительного качества.
	0.865	Не уверен, что заход на российскую территорию был запланирован. Тут или просто слегка заплутали и группа эвакуации и диверсанты.
	0.855	Западу очень стоило бы услышать сегодняшнего Лаврова: видим распространяемые у вас хотелки про заморозку боевых действий, прекрасно понимаем, что ее единственная цель - дать бандеровцам время перевооружиться и снова наброситься. Ни хрена не услышат, разумеется.
	0.854	На луганском направлении – встречные бои у Белогоровки. Северо-западнее российские войска пытаются атаковать со стороны Дибровы в сторону Заречного. А севернее – также встречные бои в Новосёловском. (Рис. 4)

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	4.667	«Западные союзники Киева выступают за поставки артиллерийских снарядов и других боеприпасов и оружия на Украину. Министр обороны Великобритании Бен Уоллес заявил, что, хотя Украина и может испытывать некоторую их нехватку, это вопрос времени. «На линии фронта всегда чувствовался дефицит, но я не вижу никаких признаков стратегической нехватки, хотя есть небольшая задержка по времени», - сказал Уоллес. Задача, добавил он, в том, чтобы украинские военные точнее использовали оружие и продолжали бы вести бои по западным методам. «Вам нужно 100 артиллерийских снарядов, чтобы взорвать российскую позицию, или всего пять? Если вы умеете стрелять точно, вам не нужно 100 снарядов», - сказал он. «У России все еще есть в распоряжении значительные силы», - также сказал Уоллес».
	4.455	Министру задали вопрос, не считает ли она, что атаки беспилотников на российские города могут привести к эскалации. "Россия напала на Украину, поэтому у Украины есть закрепленное в Уставе ООН право на защиту своей территории и своих людей", - считает Бербок. Она также утверждала, что "Россия подвергает бомбардировкам гражданские цели". "Украина защищается в рамках международного права", - полагает Бербок.
	3.489	Давайте попробуем спрогнозировать, что с Украиной будет потом. Пока руководство России не дает внятных пояснение, что правильно. Это следующий этап развития ситуации, и он во многом зависит от того, чем и как завершится этап предыдущий. Не нужно себе заранее связывать руки. Комментарии Лаврова и Захаровой предназначены для ушей «наших зарубежных партнеров» и так же вряд ли имеют какое-то отношение к нашим реальным планам. И так
	3.196	С запасами БК и вооружения в Приднестровье всё нормально - благо еще с советских времен там остались запасы и склады. Но вот как в случае чего туда перебрасывать личный состав для поддержки? Практически, никак. Разве что пройти военным маршем через всю Украину (что, безусловно, было бы красиво!).
	2.860	На луганском направлении российские войска пытались продвинуться в Серебрянском лесном массиве. При поддержке артиллерии вели наступательные действия на Терны и Невское. А значительно север-западнее атаковали в районе Синьковки. Цель – выдавить украинские войска за реку Оскол и продвинуться к стратегически важному Купянску. (Рис. 4)
Input question: Запад придерживается двойных стандартов и разжигает вооруженные конфликты по всему миру.
Top-5 lexical search (BM25) hits
	18.464	- и вот теперь ОПЕК, нефть и падение бирж по всему миру.
	17.795	— По всему миру продолжаются массовые митинги в поддержку сектора Газа.
	17.343	Это языки, на которых думают и говорят сотни миллионов людей по всему миру.
	17.318	МИД РФ: поставляемое киевскому режиму Западом оружие бесконтрольно расходится по всему миру
	16.873	В этом году мы бросили вызов всему миру.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.919	МИД РФ: поставляемое киевскому режиму Западом оружие бесконтрольно расходится по всему миру
	0.917	МВД РФ публикует видео с задержанной Дарьей Треповой
	0.914	За время проведения СВО мы видели самые разные дроны, беспилотники и квадрокоптеры всех размеров, форм и назначений. Беспилотная авиация шагнула далеко вперёд за этот короткий промежуток времени, превратившись в эффективное оружие.
	0.913	Трое высокопоставленных сотрудников МВД задержаны в Симферополе
	0.910	 Два корабля ВМФ, проходившие плановый ремонт в судоремонтном заводе им. С. Орджоникидзе в городе Севастополь и поврежденные

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.289	На Запорожском направлении ВС РФ при поддержке артиллерии давят на ВСУ на западных окраинах села Работино. Украинские штурмовые подразделения ведут аналогичные действия к северо-западу от Вербового. (Рис. 2)
	8.201	Зампред Совбеза РФ Дмитрий Медведев назвал лживыми тварями министров иностранных дел G7, которые призвали Россию подтвердить недопустимость ядерной войны.
	8.191	На Запорожском направлении ВС РФ после артобстрелов вели наступательные действия к западу от села Работино. Есть продвижение. Под ударами ВСУ в Пятихатках, Малой Токмачке и Чаровном. ВС РФ – в Копанях, Новопрокоповке и Вербовом. (Рис. 2)
	8.108	За время проведения СВО мы видели самые разные дроны, беспилотники и квадрокоптеры всех размеров, форм и назначений. Беспилотная авиация шагнула далеко вперёд за этот короткий промежуток времени, превратившись в эффективное оружие.
	7.931	• Каждый гражданин в новых регионах РФ должен чувствовать себя под защитой закона, волнующие людей вопросы должны решаться в кратчайшие сроки.
Input question: Россия всегда действует в рамках международного права и Устава ООН.
Top-5 lexical search (BM25) hits
	20.409	• Россия не будет признавать Украину страной-террористом, уподобляясь нарушителям международного права на Западе;
	19.116	— Детский фонд ООН решительно против создания зон безопасности в секторе Газа и требует полного соблюдения норм международного гуманитарного права.
	18.542	НАТО осуждает выборы в новых регионах России, считает их "нарушением международного права", несмотря на то, что эти регионы вошли в состав России по результатам референдумов в соответствии с международным правом и в рамках признанного ООН принципа права на самоопределение, заявил официальный представитель альянса Дилан Уайт.
	17.987	Министру задали вопрос, не считает ли она, что атаки беспилотников на российские города могут привести к эскалации. "Россия напала на Украину, поэтому у Украины есть закрепленное в Уставе ООН право на защиту своей территории и своих людей", - считает Бербок. Она также утверждала, что "Россия подвергает бомбардировкам гражданские цели". "Украина защищается в рамках международного права", - полагает Бербок.
	16.683	Анкара действует на 100% в рамках защиты и продвижения собственных национальных интересов.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.851	Удары стали ответом на действия йеменских вооруженных сил против Израиля и США в Красном море.
	0.848	Средства ПВО ДНР перехватили украинские ракеты РСЗО "Град" и "Ураган", обломки которых рассыпались в Киевском районе Донецка вдоль проспекта Освобождения Донбасса.
	0.846	Удар по своим: артиллерия ВСУ обстреляла посёлок Васильевка в Запорожской области, ранены гражданские.
	0.843	Артиллерия ДНР подавляет огневые точки ВСУ в районе города Авдеевка.
	0.836	Движение ВСУ: украинская армия заходит в тыл российскому гарнизону в Лимане с севера.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.635	Русская Армия продолжает вести наступательные действия на Авдеевском направлении, выжигая технику противника.
	8.602	Теперь давайте не останавливаться, пока не добьемся гражданства для всех русских. Пока не выбьем из равнодушных, или корыстных, или просто ленивых, или даже искренне ошибающихся ответственных граждан все их аргументы, почему этого делать не надо.
	8.553	Россия приостанавливает свое участие в Договоре о стратегических наступательных вооружениях, заявил Путин.
	8.540	Доктор уверен, что никак не может Россия лучше справляться с вирусом, чем Европа и США, и конечно, русские врачи с диагнозами вряд ли способны разобраться. Путают они, где Ковид, а где нет. Точнее даже скрывают.
	8.454	Она уверена, что Ротару поддерживает украинских военных и личными финансами. Она добавила, что София Ротару примет участие в большом концерте на День Независимости в столице, и опровергла слухи о возможном российском гражданстве сестры.
Input question: Россия — мирная страна и пытается решить конфликты мирным путем.
Top-5 lexical search (BM25) hits
	22.423	Она отметила, что конфликты должны решаться только мирным путем, добавив, что для европейцев было бы неприемлемо "одностороннее насильственное изменение статус-кво".
	20.745	Но главное - пока что мирным путем.
	17.811	• Мирная миссия стран Африки посетит РФ до саммита Россия — Африка, запланированного на конец июля;
	17.123	"По данным Bloomberg, Россия — самая "подсанкционная" страна в мире —
	16.615	Брюссель считает, что Россия с помощью сниженных цен для «уязвимых стран» пытается создать «новые зависимости путем обострения экономической уязвимости и ситуации глобальной продовольственной безопасности».

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.862	Всегда приятно смотреть, как работают профессионалы.
	0.843	Путин выступил против снятия моратория на смертную казнь в РФ:
	0.840	Принимаются процессуальные решения и меры профилактического характера.
	0.837	Решения по ратификации соглашений между РФ и донбасскими республиками пописаны комитетами Госдумы
	0.834	• Мирная миссия стран Африки посетит РФ до саммита Россия — Африка, запланированного на конец июля;

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	8.875	Дисциплина, контроль/штрафы и быстрая реакция/четкая работа системы здравоохранения в целом. Все, что мы видим и у нас.
	8.734	• Мирная миссия стран Африки посетит РФ до саммита Россия — Африка, запланированного на конец июля;
	8.602	Вдвое меньше стали совершать преступления дети, вся система воспитания, профессиональная ориентация, профилактика, проводимая полицейскими, дает результат.
	8.455	Материалы, собранные беспилотником, изучает полиция. Пока в отношении задержанного составлен административный протокол о нарушении режима пребывания в РФ.
	8.420	США рассматривают военные варианты противодействия РФ в сирийском небе, сообщил чиновник Пентагона.
Input question: В Украине власть захватила хунта, которая ведет агрессивную политику: обстреливает своих же граждан, притесняет церковь и российских журналистов.
Top-5 lexical search (BM25) hits
	16.669	• ряд российских военкоров и журналистов,
	16.602	 Вооружённые силы России захватила одну из танковых баз в центральной Украине.
	16.395	Народная милиция ДНР при поддержке российских военных захватила четыре танка ВСУ в окрестностях Андреевки.
	15.729	МИД Франции призвал своих граждан немедленно покинуть Беларусь в связи с военными действиями на Украине
	15.338	Ну то есть Александр Григорьевич Лукашенко взял в заложники десятки российских граждан, чтобы использовать их в своих внутриполитических целях.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.884	Вчера, в результате обстрела ВСУ группы военных корреспондентов в районе села Пятихатки в Запорожской области, унёсшего жизнь Ростислава Журавлёва, также был ранен фотокорреспондент РИА Новости Константин Михальчевский.
	0.878	Уже не раз говорилось, что украинское общество живёт внутри смеси культа смерти, националистической идеи, христианского сектантства, и все это на движке средневековья и древних архетипов.
	0.877	В городке Хуст Закарпатской области местные жены устроили митинг у здания военкомата, протестуя против отправки их мужей, добровольцев из теробороны на передовую.
	0.877	Армии же Колчака, наоборот, не хватило гибкости. Адмирал с одной стороны слишком закрутил гайки, настроив против себя сибирское крестьянство, а с другой - не мог урезонить региональных варлордов Семенова, Калмыкова и Анненкова.
	0.866	Но самое интересное, что все они в спецовочных куртках работников завода. Можно только гадать, скольким таким защитникам удалось эвакуироваться под видом мирных жителей.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.097	Важно! Поступают сообщения от местных жителей из Харьковской области. ВСУ шестой день накапливают технику и живую силу на Балаклейском направлении. Возможно готовят контрнаступление в этом районе. Возможно в направлении Изюма. И возможно вчера, украинское командование пожертвовало многими сотнями своих вояк на Херсонском направлении, чтобы отвлечь от других направлений. Да, тупо, но есть логика даже в их тупости.
	5.748	У нас же закон вроде как один для всех? Значит, повторюсь, тогда или всем по диаспоре, или государство радикально меняет отношение к функционированию национально-культурных автономий.
	5.687	И после всего этого у Зеленского хватает наглости и цинизма заявить о том, что в гибели мирных жителей виноваты российские войска! Спросите родителей Прокопенко Руслана Даниловича 13.06.2002 г.р., в дом которых, на моих глазах влетел снаряд ВСУшной гаубицы, кто помогал вывозить на кладбище и хоронить тело их сына.
	5.539	Уже не раз говорилось, что украинское общество живёт внутри смеси культа смерти, националистической идеи, христианского сектантства, и все это на движке средневековья и древних архетипов.
	5.419	— В ЦАХАЛ уверенны, что ХАМАС готовится к длительной войне. Об этом, в том числе, говорит уменьшение количества ракетных пусков по территории Израиля.
Input question: Россия показывает свою силу только только для того, чтобы защитить мирных жителей, когда мирные инструменты исчерпаны.
Top-5 lexical search (BM25) hits
	25.725	Пока только пять мирных жителей смогли
	25.279	Китай озвучил мирные предложения в Украинском кризисе в аккурат перед Мюнхеном, ТОЛЬКО ДЛЯ ТОГО, чтобы раскачать и раскочегарить Штаты, только для того, чтобы Штаты восприняли это максимально агрессивно (мол куда вы там лезете).
	22.342	И все эти комариные укусы делаются только для того, чтобы не испугать, а разозлить российский народ против власти, дескать она не может их защитить.
	21.427	А в Мариуполе, когда был согласован план и время эвакуации мирных жителей из города, когда мирные выстроились в огромные очереди для выхода, нацбаты перекрыли пути эвакуации. Целый город в заложниках.
	20.488	Решаются не только боевые задачи, но и вопросы обеспечения безопасности мирных жителей Луганской области.

-------------------------

Top-5 Bi-Encoder Retrieval hits
	0.863	После того, как Зеленский выполнил требования психа с автоматом, все подобные ему, и не только психи, станут считать, что терроризм это отличный способ добиться желаемого.
	0.860	Жуткие кадры расправы над мирными жителями, которые пытались покинуть Запорожье. Их расстреляли боевики Зеленского из полиции и теробороны.
	0.855	"К мероприятию присоединились дети, жены и близкие украинских защитников. Десятки людей собрались, чтобы выразить свою поддержку героям, которые сейчас находятся в плену страны-агрессора", — пишут местные тг-каналы.
	0.852	Землетрясения в Турции были настолько мощными, что ж/д пути между провинциями Кахраманмараш и Газиантеп теперь выглядят так.
	0.849	Проблема идеологий в том, что они навязывают людям вместо собственных интересов какие-то непонятные фантомы за которые мы все должны жить и умирать.

-------------------------

Top-5 Cross-Encoder Re-ranker hits
	6.626	Рискну предположить, что дальше выкидывать твиты русских каналов, сюжеты российских журналистов из You Tube, фильмы наших коллег, посты в Facebook будут с такими же залихватскими определениями.
	6.470	Ну а Россия же в свою очередь, более чем внятно этим ударом обозначила, что никому не стоит строить иллюзии про какие-то бесполетные зоны и тем более не стоит полагать, что есть какие-то зоны вне досягаемости ВКС.
	6.419	Российские военные в Сирии продолжают помогать местным жителям после землетрясения.
	6.311	Раскрыта тайна о невербальных жестах политиков, которые активно используются в переговорах, в том числе Зеленским и Путиным.
	6.308	После того, как Зеленский выполнил требования психа с автоматом, все подобные ему, и не только психи, станут считать, что терроризм это отличный способ добиться желаемого.

